# Notebook 2: Curate Gc Model

2a. The first section of this notebook uses the CarveMe FA1090 automated model to identify genes and their corresponding reactions that could be added to the model. Genes and reactions were verified using KEGG and published literature to identify those that had enough supporting evidence to add. If no supporting evidence cold be found, the reaction was not added. 

2b. Correcting mass/charge imbalances for the model using evidence from the literature, from BIGG, and UNIPROT as needed. 

2c. A Gc specific biomass equation is established

2d. Infeasible loops are removed

The model is then updated with annotations from modelpolisher


In [1]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import pandas as pd
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import warnings
import gc

# Using Cobrapy 0.13.0
import cobra
import cobra.test
import cobra.flux_analysis.gapfilling
from cobra import Reaction, Metabolite
from cobra.io import write_sbml_model
from cobra.flux_analysis import  flux_variability_analysis
from cobra.flux_analysis.reaction import assess_component
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
from cobra.util import solver as sutil


#Simplify reading/writing files
cwd=os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

In [2]:
#From Jenior 2021 C.diff notebooks - benchmarking C. difficile GENRES. https://github.com/csbl/Jenior_CdifficileGENRE_2021
# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [3]:
#Set media

def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-20.


In [4]:
# Read in  model
import cobra
model=read_model(cwd+'/Gc_GENRE_2022/Models/AE004969.1.json')
model.objective='Nm_Ess_biomass'

#Ecoli model NM model is based on - to be used for metabolite charges
ecoli = read_model(cwd+'/Gc_GENRE_2022/Models/ecoli.json')

#Universal model for adding reactions
universalmodel= read_model(cwd+'/Gc_GENRE_2022/Models/universal_model_cobrapy.json')

#Downloaded carvememodel for Neisseria gonorrhoeae FA1090 from PMID: 30192979 to compare annotations and add Gc specific reactions
carveme_model= cobra.io.read_sbml_model(cwd+'/Gc_GENRE_2022/Models/Neisseria_gonorrhoeae_FA_1090.xml.gz')
carvememodel=carveme_model
cobra.io.save_json_model(carveme_model, cwd+'/Gc_GENRE_2022/Models/Neisseria_gonorrhoeae_FA_1090.json')

In [5]:
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])

def rpmi_with_lactate_only(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -20.
            
def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.


In [6]:
complete(model)
model.optimize()

,fluxes,reduced_costs
12DGR120tipp,0.000000,0.000000e+00
12DGR140tipp,0.000000,0.000000e+00
12DGR141tipp,0.000000,0.000000e+00
12DGR160tipp,0.000000,0.000000e+00
12DGR161tipp,0.000000,0.000000e+00
...,...,...
XTSNtex,0.000000,0.000000e+00
XYLtex,0.000000,0.000000e+00
XYLUtex,0.000000,0.000000e+00
ZN2abcpp,0.013314,-1.951564e-18


## 2a. Identify Genes and Reactions to Add from CarveME

In [7]:
#Identify genes/reactions to add to the model from carveme
model_genes  = []
for gene in model.genes:
    model_genes.append(gene.id)

#Get list of genes from Carveme model for FA1090 in NGO format
carveme_model_genes= list()
for gene in carveme_model.genes:
    carveme_model_genes.append(gene.id)
carveme_model_genes

#Use look_back table of annotations from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5399841/
annotation_table = pd.read_csv (cwd+'/Gc_GENRE_2022/Curate_Gc_Model/Annotation_resources/look_back_table.csv', usecols= ['Locus Tag ID','Protein.ID'])
annotation_table=annotation_table.rename(columns={'Protein.ID': 'ProteinID', 'Locus Tag ID': "NGO_ID"})
annotation_table
                        
#Match id format between kegg and model
annotation_table['ProteinID'] = annotation_table['ProteinID'].str.replace('.', '_')

annotatedgenes= annotation_table[annotation_table.ProteinID.isin(carveme_model_genes)]
converted_carveme_model_genes= list()
for tag in annotatedgenes['NGO_ID']:
        converted_carveme_model_genes.append(tag)
        
unique_genes_carveme = set(converted_carveme_model_genes) - set(model_genes) # correct elements, but not yet in sorted order
print(sorted(unique_genes_carveme)) # sort and print



['NGO0057', 'NGO0059', 'NGO0063', 'NGO0084', 'NGO0085', 'NGO0114', 'NGO0176', 'NGO0177', 'NGO0186', 'NGO0188', 'NGO0189', 'NGO0199', 'NGO0231', 'NGO0294', 'NGO0351', 'NGO0382', 'NGO0387', 'NGO0388', 'NGO0541', 'NGO0567', 'NGO0571', 'NGO0604', 'NGO0608', 'NGO0632', 'NGO0633', 'NGO0652', 'NGO0653', 'NGO0659', 'NGO0676', 'NGO0718', 'NGO0763', 'NGO0829', 'NGO0830', 'NGO0842', 'NGO0858', 'NGO0886', 'NGO0902', 'NGO0904', 'NGO0905', 'NGO0906', 'NGO0934', 'NGO0941', 'NGO0974', 'NGO1032', 'NGO1033', 'NGO1047', 'NGO1052', 'NGO1055', 'NGO1154', 'NGO1204', 'NGO1207', 'NGO1226', 'NGO1227', 'NGO1243', 'NGO1298', 'NGO1323', 'NGO1325', 'NGO1334', 'NGO1364', 'NGO1365', 'NGO1381', 'NGO1404', 'NGO1432', 'NGO1433', 'NGO1434', 'NGO1454', 'NGO1530', 'NGO1559', 'NGO1577', 'NGO1684', 'NGO1686', 'NGO1692', 'NGO1722', 'NGO1732', 'NGO1733', 'NGO1753', 'NGO1769', 'NGO1858', 'NGO1866', 'NGO1922', 'NGO1930', 'NGO1963', 'NGO1977', 'NGO1990', 'NGO2011', 'NGO2012', 'NGO2013', 'NGO2014', 'NGO2015', 'NGO2027', 'NGO2033'

<ipython-input-7-1829916bf6eb>:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  annotation_table['ProteinID'] = annotation_table['ProteinID'].str.replace('.', '_')


In [8]:
#Add NGOIDs to the carveme model as an annotation
for gene in annotatedgenes['ProteinID']:
    if gene in carveme_model.genes:
        row=annotatedgenes.index[annotatedgenes['ProteinID']==gene]
        id=annotatedgenes['NGO_ID']
        annotation =id.loc[row].item()
        carveme_model.genes.get_by_id(gene).annotation['NGO_ID']= annotation

In [9]:
carvemegenes = []
carvemengoids=[]
carvemereactions = []


for gene in carveme_model.genes:
    carvemegenes.append(gene.id)
    carvemengoids.append(gene.annotation)
    sets=list(gene.reactions)
    rxnlist=[]
    for x in sets:
        rxnlist.append(x)
    carvemereactions.append(rxnlist)

dict = {'carvemegenes': carvemegenes, 'NGO_IDs':carvemengoids,'carvemereactions':carvemereactions}
carvemegenes_reactions_DF = pandas.DataFrame(dict)
carvemegenes_reactions_DF.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/Annotation_resources/carveme_genes_reactions.csv')
carvemegenes_reactions_DF

,carvemegenes,NGO_IDs,carvemereactions
0,YP_208072_1,{'NGO_ID': 'NGO0974'},[2AGPGAT120: 2agpg120_c + atp_c + ddca_c --> a...
1,YP_208834_1,{'NGO_ID': 'NGO1804'},[C160SN: actACP_c + 20.0 h_c + 6.0 malACP_c + ...
2,YP_209169_1,{'NGO_ID': 'NGO2163'},[C160SN: actACP_c + 20.0 h_c + 6.0 malACP_c + ...
3,YP_208796_1,{'NGO_ID': 'NGO1763'},[C160SN: actACP_c + 20.0 h_c + 6.0 malACP_c + ...
4,YP_209174_1,{'NGO_ID': 'NGO2168'},[3OAS160: h_c + malACP_c + myrsACP_c --> 3opal...
...,...,...,...
505,YP_207518_1,{'NGO_ID': 'NGO0362'},[UPPDC1: 4.0 h_c + uppg3_c --> 4.0 co2_c + cpp...
506,YP_207509_1,{'NGO_ID': 'NGO0353'},[UPPRT: prpp_c + ura_c --> ppi_c + ump_c]
507,YP_207732_1,{'NGO_ID': 'NGO0589'},"[URAt: ura_e <=> ura_c, URAt2: h_e + ura_e -->..."
508,YP_208502_1,{'NGO_ID': 'NGO1448'},[USHD: h2o_c + u23ga_c --> 2.0 h_c + lipidX_c ...


In [10]:
carveme_model.metabolites.ala__L_c.id ='ala_L_c'
carveme_model.metabolites.glu__L_c.id ='glu_L_c'

In [11]:
ALATA_L =carvememodel.reactions.get_by_id('ALATA_L')
carvememodel.reactions.get_by_id('ALATA_L').gene_reaction_rule='NGO1047'
model.add_reactions([ALATA_L])
model.reactions.get_by_id('ALATA_L')

Reaction identifier,ALATA_L
Name,L-alanine transaminase
Memory address,0x020f4c812c40
Stoichiometry,akg_c + ala_L_c <=> glu_L_c + pyr_c 2-Oxoglutarate + L-Alanine <=> L-Glutamate + Pyruvate
GPR,NGO1047
Lower bound,-1000.0
Upper bound,1000.0


In [12]:
AP4AH =carvememodel.reactions.get_by_id('AP4AH')
carvememodel.reactions.get_by_id('AP4AH').gene_reaction_rule='NGO0231'
model.add_reactions([AP4AH])
model.reactions.get_by_id('AP4AH')


Reaction identifier,AP4AH
Name,Ap4A hydrolase
Memory address,0x020f4cf2ad30
Stoichiometry,"ap4a_c + h2o_c --> 2.0 adp_c + 2.0 h_c P1,P4-Bis(5'-adenosyl) tetraphosphate + H2O --> 2.0 ADP + 2.0 H"
GPR,NGO0231
Lower bound,0.0
Upper bound,1000.0


In [13]:
#ANS already exists in annotated Gc model, the model GPR should require NGO9872 and NGO1204
print (model.reactions.get_by_id('ANS').gene_reaction_rule)
model.reactions.get_by_id('ANS').gene_reaction_rule='NGO0872 and NGO1204'
print (model.reactions.get_by_id('ANS').gene_reaction_rule)

NGO0872
NGO0872 and NGO1204


In [14]:
BTDD_RR =carvememodel.reactions.get_by_id('BTDD_RR')
carvememodel.reactions.get_by_id('BTDD_RR').gene_reaction_rule='NGO0186'
model.add_reactions([BTDD_RR])
model.reactions.get_by_id('BTDD_RR')

Reaction identifier,BTDD_RR
Name,R R butanediol dehydrogenase
Memory address,0x020f4d204ac0
Stoichiometry,btd_RR_c + nad_c <=> actn__R_c + h_c + nadh_c R R 2 3 Butanediol C4H10O2 + Nicotinamide-adenine-dinucleotide <=> R Acetoin C4H8O2 + H + Nicotinamide-adenine-dinucleotide--reduced
GPR,NGO0186
Lower bound,-1000.0
Upper bound,1000.0


In [15]:
CYTOM =carvememodel.reactions.get_by_id('CYTOM')
carvememodel.reactions.get_by_id('CYTOM').gene_reaction_rule='NGO0676'
model.add_reactions([CYTOM])
model.reactions.get_by_id('CYTOM')

Reaction identifier,CYTOM
Name,Cytosine 5 methyltransferase
Memory address,0x020f4aeeaa00
Stoichiometry,amet_c + csn_c <=> 5mcsn_c + ahcys_c + h_c S-Adenosyl-L-methionine + Cytosine <=> 5-Methylcytosine + S-Adenosyl-L-homocysteine + H
GPR,NGO0676
Lower bound,-1000.0
Upper bound,1000.0


In [16]:
#The gene currently indicated for this reaction is NGO1506 - however KEGG annotates this gene as a 8-oxo-dGTP diphosphatase (MutT), KEGG reaction 	 R09832
#CARVEME indicates this reactions is produced by NGO1334. However this gene is annotated as a putative RNA pyrophosphohydrolase. The function isn't clear. 
# Replacing with the KEGG annotated gene for this reaction NGO0224

DNTPPA =carvememodel.reactions.get_by_id('DNTPPA')
model.reactions.get_by_id('DNTPPA').gene_reaction_rule='NGO0224'
model.reactions.get_by_id('DNTPPA')


Reaction identifier,DNTPPA
Name,R_DNTPPA
Memory address,0x020f45d0aeb0
Stoichiometry,ahdt_c + h2o_c --> dhpmp_c + h_c + ppi_c 2-Amino-4-hydroxy-6-erythro-1-2-3-trihydroxypropyl-dihydropteridine-triphosphate + H2O --> Dihydroneopterin-monophosphate + H + Diphosphate
GPR,NGO0224
Lower bound,0.0
Upper bound,999999.0


In [17]:
#DNMPPA is not annotated in KEGG, but is predicted to use an enzyme with E.C. number 3.6.1-, and is listed as an orphan gene in the current model. 
#I am tentatively de-orphaning the gene by adding the annotation from CARVEME for NGO1334

DNMPPA =carvememodel.reactions.get_by_id('DNMPPA')
model.reactions.get_by_id('DNMPPA').gene_reaction_rule='NGO1334'
model.reactions.get_by_id('DNMPPA')

Reaction identifier,DNMPPA
Name,R_DNMPPA
Memory address,0x020f45d0a940
Stoichiometry,dhpmp_c + h2o_c --> dhnpt_c + pi_c Dihydroneopterin-monophosphate + H2O --> Dihydroneopterin + Phosphate
GPR,NGO1334
Lower bound,0.0
Upper bound,999999.0


In [18]:
#CARVEME indicates FADRx uses NGO0059 or NGO0659. However NGO0659 has different annotations in KEGG that do not match this function. Removed NGO0659. 
FADRx =carvememodel.reactions.get_by_id('FADRx')
carvememodel.reactions.get_by_id('FADRx').gene_reaction_rule='NGO0059'
model.add_reactions([FADRx])
model.reactions.get_by_id('FADRx')

Reaction identifier,FADRx
Name,FAD reductase
Memory address,0x020f4b077760
Stoichiometry,fad_c + h_c + nadh_c --> fadh2_c + nad_c Flavin-adenine-dinucleotide-oxidized + H + Nicotinamide-adenine-dinucleotide--reduced --> Flavin-adenine-dinucleotide-reduced + Nicotinamide-adenine-dinucleotide
GPR,NGO0059
Lower bound,0.0
Upper bound,1000.0


In [19]:
#CARVEME and KEGG indicate presence of this reaction
HPA3MO =carvememodel.reactions.get_by_id('HPA3MO')
carvememodel.reactions.get_by_id('HPA3MO').gene_reaction_rule='NGO0059'
model.add_reactions([HPA3MO])
model.reactions.get_by_id('HPA3MO')

Reaction identifier,HPA3MO
Name,4 hydroxyphenylacetate 3 monooxygenase nadh
Memory address,0x020f5366b4f0
Stoichiometry,4hphac_c + h_c + nadh_c + o2_c --> 34dhpha_c + h2o_c + nad_c 4-Hydroxyphenylacetate + H + Nicotinamide-adenine-dinucleotide--reduced + O2 --> 3-4-Dihydroxyphenylacetate + H2O + Nicotinamide-adenine-dinucleotide
GPR,NGO0059
Lower bound,0.0
Upper bound,1000.0


In [20]:
#Model has NGO1406 annotated for GLYCL. CARVEME has NGO1325 annotated. 
#The Glycine cleavage system is a functional system composed of 4 different components *H, P, T, and L. https://www.biocyc.org/GCF_000006845/NEW-IMAGE?type=PATHWAY&object=GLYCLEAV-PWY
#All 4 components should be required for function of this system. As such, I am listing ALL 6 genes, indicated in KEGG, for this reaction. 

GLYCL =carvememodel.reactions.get_by_id('GLYCL')
model.reactions.get_by_id('GLYCL').gene_reaction_rule='NGO0562 and NGO0915 and NGO0925 and NGO1325 and NGO1404 and NGO1406'
model.reactions.get_by_id('GLYCL')

Reaction identifier,GLYCL
Name,R_GLYCL
Memory address,0x020f45e02eb0
Stoichiometry,gly_c + nad_c + thf_c <=> co2_c + mlthf_c + nadh_c + nh4_c Glycine + Nicotinamide-adenine-dinucleotide + 5-6-7-8-Tetrahydrofolate <=> CO2 + 5-10-Methylenetetrahydrofolate + Nicotinamide-adenine-dinucleotide--reduced + Ammonium
GPR,NGO0562 and NGO0915 and NGO0925 and NGO1325 and NGO1404 and NGO1406
Lower bound,-999999.0
Upper bound,999999.0


In [21]:
#CARVEME indicated NGO2033
#KEGG indicates NGO1187 AND NGO2033 participate in this reaction https://www.genome.jp/entry/ngo:NGO_1187+ngo:NGO_2033


GLYOX =carvememodel.reactions.get_by_id('GLYOX')
carvememodel.reactions.get_by_id('GLYOX')

Reaction identifier,GLYOX
Name,Hydroxyacylglutathione hydrolase
Memory address,0x020f4eff9d60
Stoichiometry,h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c H2O + (R)-S-Lactoylglutathione --> Reduced glutathione + H+ + D-Lactate
GPR,YP_208259_1 or YP_209053_1
Lower bound,0.0
Upper bound,1000.0


In [22]:
model.reactions.get_by_id('HPA3MO').gene_reaction_rule='NGO1187 and NGO2033'
model.reactions.get_by_id('GLYOX')

Reaction identifier,GLYOX
Name,R_GLYOX
Memory address,0x020f45e0e7f0
Stoichiometry,h2o_c + lgt_S_c <=> gthrd_c + h_c + lac_D_c H2O + R-S-Lactoylglutathione <=> Reduced-glutathione + H + D-Lactate
GPR,NGO1187
Lower bound,-999999.0
Upper bound,999999.0


In [23]:
#CARVEME indicates NGO0387 as performing this reaction, and so does KEGG. https://www.genome.jp/entry/ngo:NGO_0387
GTPCI =carvememodel.reactions.get_by_id('GTPCI')
model.reactions.get_by_id('GTPCI').gene_reaction_rule='NGO0387'
model.reactions.get_by_id('GTPCI')

Reaction identifier,GTPCI
Name,R_GTPCI
Memory address,0x020f45e1bca0
Stoichiometry,gtp_c + h2o_c --> ahdt_c + for_c + h_c GTP + H2O --> 2-Amino-4-hydroxy-6-erythro-1-2-3-trihydroxypropyl-dihydropteridine-triphosphate + Formate + H
GPR,NGO0387
Lower bound,0.0
Upper bound,999999.0


In [24]:
#Adding reactions according to PMCID: PMC4697154/PMID: 26567338 which are conducted by plsX (Ngo2151)
x = ['ACPPAT161' , 'G3PAT120' , 'ACPPAT181' , 'G3PAT180' , 'ACPPAT120' , 'G3PAT181' , 'ACPPAT141' , 'G3PAT140' , 'ACPPAT160' , 'G3PAT161' , 'G3PAT160' , 'ACPPAT140' , 'ACPPAT180']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO2171'
    model.add_reactions([reaction])

Ignoring reaction 'G3PAT120' since it already exists.
Ignoring reaction 'G3PAT180' since it already exists.
Ignoring reaction 'G3PAT181' since it already exists.
Ignoring reaction 'G3PAT140' since it already exists.
Ignoring reaction 'G3PAT160' since it already exists.


In [25]:
#providing gene for orphan reactions according to PMCID: PMC4697154/PMID: 26567338 which are conducted by plsX (Ngo2151)
x = ['G3PAT120' , 'G3PAT180' , 'G3PAT181' , 'G3PAT140' , 'G3PAT161' , 'G3PAT160' ]
for r in x:
    reaction =model.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO2171'


In [26]:
#Adding reactions according to PMCID: PMC4697154/PMID: 26567338 which are conducted by plsY (Ngo2151)
x = ['APG3PAT181' , 'APG3PAT160' , 'APG3PAT180' , 'APG3PAT120' , 'APG3PAT141' , 'APG3PAT161' , 'APG3PAT140']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0858'
    model.add_reactions([reaction])


In [27]:
#Adding peptidase reaction
x = ['AMPTASEPG']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO1227'
    model.add_reactions([reaction])

In [28]:
#cardiolipin has been in cultures of Gc PMCID: PMC235958/ PMID: 810478
x = ['CLPNS181pp' , 'CLPNS141pp' , 'CLPNS140pp' , 'CLPNS160pp' , 'CLPNS120pp' , 'CLPNS161pp' , 'CLPNS180pp']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0902'
    model.add_reactions([reaction])

In [29]:
#reduction of H2O2 to water facilitated by cytochrome c peroxidase, with Laz as an electron donor. Laz is encoded by NGO0994
reaction = cobra.Reaction('CYPOX')
reaction.name = 'Cytochrome C Peroxidase'
reaction.lower_bound = 0.  
reaction.upper_bound = 1000.  
reaction.gene_reaction_rule='NGO1769 and NGO0994'



h_c=model.metabolites.get_by_id('h_c')
h2o2= model.metabolites.get_by_id('h2o2_c')
h2o_c=model.metabolites.get_by_id('h2o_c')

reaction.add_metabolites({
    h2o2: -1.0,
    h_c: -2.0,
    h2o_c: 2.0,
})

model.add_reactions([reaction])
print(model.reactions.CYPOX)
print(model.reactions.CYPOX.check_mass_balance())

CYPOX: h2o2_c + 2.0 h_c --> 2.0 h2o_c
{}


In [30]:
x = ['FLVR', 'FLVRx']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0388'
    model.add_reactions([reaction])

In [31]:
#This reaction is indicated as a bidirectional reaction in Bigg. However Kegg indicates it is 
#unidirection with NGO1977 performing the forward reaction and NGO1406 the reverse reaction (https://www.kegg.jp/pathway/ngo00670+NGO_1977)
x = ['FOMETRi']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO1977 and NGO1406'
    model.add_reactions([reaction])

In [32]:
model.reactions.GLYCLTDx.gene_reaction_rule='NGO2043 or NGO1243'

In [33]:
x = ['GRXR']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0114 or NGO1381 or NGO0351'
    model.add_reactions([reaction])

In [34]:
#Carveme did not correctly annotate NGO1684, (its Not GTPCI) but it does perform a reaction just downstream. 
#Involved in folate biosynthesis. Corrected and added the full pathway.
#To include NGO0387 (GTPCI), NGO0131, NGO00720, NGO0129, NGO1684 (https://www.genome.jp/pathway/ngo00790+NGO_1684)
x = ['PTHPS']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0131'
    model.add_reactions([reaction])
x = ['CPH4S']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0131'
    model.add_reactions([reaction])
x = ['CDGS']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO00720'
    model.add_reactions([reaction])
x = ['CDGR']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO1684'
    model.add_reactions([reaction])
x = ['CCGS']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0129'
    model.add_reactions([reaction])

In [35]:
model.reactions.Kt2pp.gene_reaction_rule='NGO1774 or NGO1154'

In [36]:
x = ['MLDEP2pp']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0571 or NGO1686'
    model.add_reactions([reaction])
#ltgD (0626) and ltgC (2048) ltgB (1033) and ltgE(0608) are all murein lytic transcglycosylases. PMCID: PMC3412582/PMID: 22432703
#Adding all of them as potential genes 
x = ['MLTGY3pp']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0608 or NGO1033 or NGO0626 or NGO2048'
    model.add_reactions([reaction])

x = ['MLTGY1pp' ,'MLTGY4pp' , 'MLTGY2pp']
for r in x:
    reaction =model.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0608 or NGO1033 or NGO0626 or NGO2048'

In [37]:
#Deorphan PPPGO
model.reactions.PPPGO.gene_reaction_rule= 'NGO1692'

In [38]:
#Add additional thioredoxin proteins
model.reactions.THIORDXi.gene_reaction_rule=  'NGO0328 or NGO1923 or NGO0652 or NGO0057'

In [39]:
#CARVEME indicates NGO1052
#KEGG indicates NGO0073 and NGO1052
#NGO0203 has a different EC number - EC 3.1.3.105 . I am replacing with the KEGG/CARVEME gene annotations

PGLYCP =carvememodel.reactions.get_by_id('PGLYCP')

model.reactions.get_by_id('PGLYCP').gene_reaction_rule='NGO0073 or NGO1052'
model.reactions.get_by_id('PGLYCP')

Reaction identifier,PGLYCP
Name,R_PGLYCP
Memory address,0x020f45f77c70
Stoichiometry,2pglyc_c + h2o_c --> glyclt_c + pi_c 2-Phosphoglycolate + H2O --> Glycolate + Phosphate
GPR,NGO0073 or NGO1052
Lower bound,0.0
Upper bound,999999.0


In [40]:
#CARVEME indicates NGO0435 or NGO1752. Model has mislabeled NGO number with an underscore. Replacing.
SPMS =carvememodel.reactions.get_by_id('SPMS')
model.reactions.get_by_id('SPMS').gene_reaction_rule='NGO0435 or NGO1752'
model.reactions.get_by_id('SPMS')

Reaction identifier,SPMS
Name,R_SPMS
Memory address,0x020f46060a60
Stoichiometry,ametam_c + ptrc_c --> 5mta_c + h_c + spmd_c S-Adenosylmethioninamine + Putrescine --> 5-Methylthioadenosine + H + Spermidine
GPR,NGO0435 or NGO1752
Lower bound,0.0
Upper bound,999999.0


In [41]:
#Model contains ethanol production - however Gc does not produce ethanol. Removing this reaction.
ALCD2x =model.reactions.get_by_id('ALCD2x')
model.remove_reactions([ALCD2x])



In [42]:
#Similarly, the mening model had added acetylaldehyde production, without genetic evidence, to support the production of ethanol observed in mening. 
#Because Gc does not produce ethanol or acetylaldehyde, removing these associated reaction.
ALDD2y =model.reactions.get_by_id('ALDD2y')
model.remove_reactions([ALDD2y])

ACALDtpp =model.reactions.get_by_id('ACALDtpp')
model.remove_reactions([ACALDtpp])

EX_acald_e_ =model.reactions.get_by_id('EX_acald_e_')
model.remove_reactions([EX_acald_e_])

In [43]:
rpmi(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_e,EX_arg_L_e_,0.2866,6,1.19%
asn_L_e,EX_asn_L_e_,0.2159,4,0.60%
asp_L_e,EX_asp_L_e_,3.727,4,10.35%
ca2_e,EX_ca2_e_,0.004065,0,0.00%
cl_e,EX_cl_e_,0.004065,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.00271,0,0.00%
cu2_e,EX_cu2_e_,0.00271,0,0.00%
cys_L_e,EX_cys_L_e_,3.235,3,6.74%
fe3_e,EX_fe3_e_,0.01258,0,0.00%
glc_D_e,EX_glc_D_e_,10,6,41.64%


In [44]:
#CARVEME revealed that NGO0904-NGO0906 is not present in the Gc model - which was recently discovered as an Gc
#lactate dehydrogenase. Additionally other aspects of lactate metabolism are corrected here. 

#PMCID: PMC4215069
#PMCID: PMC7530546

In [45]:
model.metabolites.get_by_id('lac_D_c')

Metabolite identifier,lac_D_c
Name,D-Lactate
Memory address,0x020f3d0c81c0
Formula,C3H5O3
Compartment,Cytosol
In 3 reaction(s),"DLACt2pp, LDH_D, GLYOX"


In [46]:
carvememodel.metabolites.get_by_id('lac__D_c').id='lac_D_c'

In [47]:
model.reactions.get_by_id('L_LACD4')

Reaction identifier,L_LACD4
Name,R_L_LACD4
Memory address,0x020f45e67f40
Stoichiometry,lac_L_c + nad_c --> h_c + nadh_c + pyr_c L-Lactate + Nicotinamide-adenine-dinucleotide --> H + Nicotinamide-adenine-dinucleotide--reduced + Pyruvate
GPR,NGO0639 or Orphan
Lower bound,0.0
Upper bound,999999.0


In [48]:
#NGO0639 is a quinone depdendent L-lactate dehydrogenase. There are no known NAD/NADH dependent L-lactate dehydrogenases in Gc.
LACD4 =model.reactions.get_by_id('L_LACD4')
model.remove_reactions([LACD4])
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_e,EX_arg_L_e_,0.1354,6,1.03%
asn_L_e,EX_asn_L_e_,0.102,4,0.52%
asp_L_e,EX_asp_L_e_,0.2643,4,1.34%
ca2_e,EX_ca2_e_,0.00192,0,0.00%
cl_e,EX_cl_e_,0.00192,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.00128,0,0.00%
cu2_e,EX_cu2_e_,0.00128,0,0.00%
cys_L_e,EX_cys_L_e_,0.03201,3,0.12%
fe3_e,EX_fe3_e_,0.005943,0,0.00%
glu_L_e,EX_glu_L_e_,0.2879,5,1.83%


In [49]:
#The reaction LDH_D is a the NAD dependent conversion of D_lac to pyruvate. It is completed by ldhA (NGO1336). 
#This reaction should be bidirectional. 

model.reactions.get_by_id('LDH_D')
model.reactions.get_by_id('LDH_D').gene_reaction_rule='NGO1336'
model.reactions.get_by_id('LDH_D').lower_bound =-1000.
model.reactions.get_by_id('LDH_D')

Reaction identifier,LDH_D
Name,R_LDH_D
Memory address,0x020f45e73760
Stoichiometry,lac_D_c + nad_c <=> h_c + nadh_c + pyr_c D-Lactate + Nicotinamide-adenine-dinucleotide <=> H + Nicotinamide-adenine-dinucleotide--reduced + Pyruvate
GPR,NGO1336
Lower bound,-1000.0
Upper bound,999999.0


In [50]:
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_e,EX_arg_L_e_,0.1354,6,1.03%
asn_L_e,EX_asn_L_e_,0.102,4,0.52%
asp_L_e,EX_asp_L_e_,0.2643,4,1.34%
ca2_e,EX_ca2_e_,0.00192,0,0.00%
cl_e,EX_cl_e_,0.00192,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.00128,0,0.00%
cu2_e,EX_cu2_e_,0.00128,0,0.00%
cys_L_e,EX_cys_L_e_,0.03201,3,0.12%
fe3_e,EX_fe3_e_,0.005943,0,0.00%
glu_L_e,EX_glu_L_e_,0.2879,5,1.83%


In [51]:
#The reaction LDH_D2 is competed by ldhD (NGO0890), a Quinone dependent D-lactate dehydrogenase.
LDH_D2=carvememodel.reactions.get_by_id('LDH_D2')
carvememodel.reactions.get_by_id('LDH_D2').gene_reaction_rule='NGO0890'
model.add_reactions([LDH_D2])
model.reactions.get_by_id('LDH_D2')


Reaction identifier,LDH_D2
Name,D-lactate dehydrogenase
Memory address,0x020f5a320be0
Stoichiometry,lac_D_c + q8_c --> pyr_c + q8h2_c D-Lactate + Ubiquinone-8 --> Pyruvate + Ubiquinol-8
GPR,NGO0890
Lower bound,0.0
Upper bound,1000.0


In [52]:
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_e,EX_arg_L_e_,0.1354,6,1.03%
asn_L_e,EX_asn_L_e_,0.102,4,0.52%
asp_L_e,EX_asp_L_e_,0.2643,4,1.34%
ca2_e,EX_ca2_e_,0.00192,0,0.00%
cl_e,EX_cl_e_,0.00192,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.00128,0,0.00%
cu2_e,EX_cu2_e_,0.00128,0,0.00%
cys_L_e,EX_cys_L_e_,0.03201,3,0.12%
fe3_e,EX_fe3_e_,0.005942,0,0.00%
glu_L_e,EX_glu_L_e_,0.2879,5,1.83%


In [53]:
#L_LACD2 is done by lldD (NGO0639), and l-lactate dehydrogenase that is quinone dependent. 
#IT was also found that lutCAB (NGO0904-NGO0906) are also L-lactate dehydrogenases, predicted to be quinone dependent.
# These are added here. 
model.reactions.get_by_id('L_LACD2').gene_reaction_rule='NGO0639 or (NGO0904 and NGO0905 and NGO0906)'
model.reactions.get_by_id('L_LACD2')



Reaction identifier,L_LACD2
Name,R_L_LACD2
Memory address,0x020f45e67fd0
Stoichiometry,lac_L_c + q8_c --> pyr_c + q8h2_c L-Lactate + Ubiquinone-8 --> Pyruvate + Ubiquinol-8
GPR,NGO0639 or (NGO0904 and NGO0905 and NGO0906)
Lower bound,0.0
Upper bound,999999.0


In [54]:
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_e,EX_arg_L_e_,0.1354,6,1.03%
asn_L_e,EX_asn_L_e_,0.102,4,0.52%
asp_L_e,EX_asp_L_e_,0.2643,4,1.34%
ca2_e,EX_ca2_e_,0.00192,0,0.00%
cl_e,EX_cl_e_,0.00192,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.00128,0,0.00%
cu2_e,EX_cu2_e_,0.00128,0,0.00%
cys_L_e,EX_cys_L_e_,0.03201,3,0.12%
fe3_e,EX_fe3_e_,0.005942,0,0.00%
glu_L_e,EX_glu_L_e_,0.2879,5,1.83%


In [55]:
model.genes.get_by_id('NGO1449')

Gene identifier,NGO1449
Name,
Memory address,0x020f45b8b430
Functional,True
In 3 reaction(s),"DLACt2pp, GLYCLTt2rpp, LLACt3pp"


In [56]:
#L-LActate transport is abolished in an lctP (NGO1449) mutant, therefore, removing NGO1361 from L-lactate transport.
#NGO1361 is homologous to the L-lactate transporter lldP from E. coli, however it is much shorter in Gc, only 71 amino acids. It appears to be truncated/nonfunctioal. 
# So I am removing it also from DLACt2pp transport.

model.reactions.get_by_id('LLACt3pp')
model.reactions.get_by_id('LLACt3pp').gene_reaction_rule='NGO1449'
model.reactions.get_by_id('LLACt3pp')

model.reactions.get_by_id('DLACt2pp')
model.reactions.get_by_id('DLACt2pp').gene_reaction_rule='NGO1449'
model.reactions.get_by_id('DLACt2pp')



Reaction identifier,DLACt2pp
Name,R_DLACt2pp
Memory address,0x020f45cff310
Stoichiometry,h_p + lac_D_p <=> h_c + lac_D_c H + D-Lactate <=> H + D-Lactate
GPR,NGO1449
Lower bound,-999999.0
Upper bound,999999.0


In [57]:
rpmi(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg_L_e,EX_arg_L_e_,0.2866,6,1.19%
asn_L_e,EX_asn_L_e_,0.2159,4,0.60%
asp_L_e,EX_asp_L_e_,3.727,4,10.35%
ca2_e,EX_ca2_e_,0.004065,0,0.00%
cl_e,EX_cl_e_,0.004065,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.00271,0,0.00%
cu2_e,EX_cu2_e_,0.00271,0,0.00%
cys_L_e,EX_cys_L_e_,3.235,3,6.74%
fe3_e,EX_fe3_e_,0.01258,0,0.00%
glc_D_e,EX_glc_D_e_,10,6,41.64%


In [58]:
model.reactions.get_by_id('UCYR')

Reaction identifier,UCYR
Name,R_UCYR
Memory address,0x020f460aadc0
Stoichiometry,"2.0 focytC_p + q8h2_c <=> 2.0 ficytC_p + 2.0 h_p + q8_c 2.0 ferrocytochrome, cytc + Ubiquinol-8 <=> 2.0 ferricytochrome, cytc + 2.0 H + Ubiquinone-8"
GPR,NGO2031 and NGO2030 and NGO2029 and (NGO1080 or NGO0101 or Orphan)
Lower bound,-999999.0
Upper bound,999999.0


In [59]:
model.metabolites.get_by_id('q8_c')

Metabolite identifier,q8_c
Name,Ubiquinone-8
Memory address,0x020f45aa3ee0
Formula,C49H74O4
Compartment,Cytosol
In 12 reaction(s),"FUMR, NADHQNa, NADH5, MDH2, NADPHQR2, NADH16pp, DHORD2, SUCD1, ASPO3, UCYR, L_LACD2, LDH_D2"


In [60]:
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])

def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0. 



mdm_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_co2_e_', 'EX_cobalt2_e_', 'EX_cu2_e_' , 'EX_h_e_', 'EX_h2o_e_','EX_mn2_e_','EX_mobd_e_','EX_tungs_e_', 'EX_zn2_e_','EX_na1_e_','EX_cl_e_','EX_k_e_','EX_so4_e_','EX_nh4_e_','EX_mg2_e_','EX_pi_e_','EX_ca2_e_','EX_fe3_e_','EX_no3_e_','EX_asp_L_e_','EX_glu_L_e_','EX_arg_L_e_','EX_gly_e_','EX_ser_L_e_','EX_leu_L_e_','EX_ile_L_e_','EX_val_L_e_','EX_tyr_L_e_','EX_cys_L_e_','EX_pro_L_e_','EX_trp_L_e_','EX_thr_L_e_','EX_phe_L_e_','EX_asn_L_e_','EX_gln_L_e_','EX_his_L_e_','EX_met_L_e_','EX_ala_L_e_','EX_lys_L_e_','EX_gthrd_e_','EX_thm_e_','EX_pnto_R_e_','EX_glc_D_e_','EX_hxan_e_','EX_ura_e_'])

def mdm(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0.        

def mdm_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.              

def mdm_with_lactate_only(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10. 
            
            
def mdm_molarity_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -27.8
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -20
    

def mdm_molarity_with_lactate_only(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -0
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -20
               
def mdm_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -27.8
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0
                         
        

In [61]:
for reaction in model.reactions:
     if reaction.lower_bound < -4:
            reaction.lower_bound=-1000.
for reaction in model.reactions:
     if reaction.upper_bound > 4:
            reaction.upper_bound=1000.

In [62]:
mdm(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ala_L_e,EX_ala_L_e_,10,3,14.86%
arg_L_e,EX_arg_L_e_,0.3144,6,0.93%
asn_L_e,EX_asn_L_e_,0.2369,4,0.47%
asp_L_e,EX_asp_L_e_,8.49,4,16.82%
ca2_e,EX_ca2_e_,0.004459,0,0.00%
cl_e,EX_cl_e_,0.004459,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.002973,0,0.00%
cu2_e,EX_cu2_e_,0.002973,0,0.00%
cys_L_e,EX_cys_L_e_,8.197,3,12.18%
fe3_e,EX_fe3_e_,0.0138,0,0.00%


In [63]:
mdm_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ala_L_e,EX_ala_L_e_,10,3,9.75%
arg_L_e,EX_arg_L_e_,0.764,6,1.49%
asn_L_e,EX_asn_L_e_,0.5756,4,0.75%
ca2_e,EX_ca2_e_,0.01084,0,0.00%
cl_e,EX_cl_e_,0.01084,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.007225,0,0.00%
cu2_e,EX_cu2_e_,0.007225,0,0.00%
cys_L_e,EX_cys_L_e_,0.18,3,0.18%
fe3_e,EX_fe3_e_,0.03354,0,0.00%
gln_L_e,EX_gln_L_e_,10,5,16.26%


In [64]:
model

Name,AE004969.1
Memory address,0x020f3d096130
Number of metabolites,1334
Number of reactions,1521
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"Cytosol, C_c, C_p, c"


In [65]:
len(model.genes)

554

In [66]:
model

Name,AE004969.1
Memory address,0x020f3d096130
Number of metabolites,1334
Number of reactions,1521
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"Cytosol, C_c, C_p, c"


## Add Additional Curation from literature for Gc

In [67]:
#The cytochrome oxidation was flipped in the original CYOX reaction. Correcting here. 
#based on other examples from Bigg http://bigg.ucsd.edu/models/iAF987/reactions/CYOO2pp
#Additionally, no hydrogens were exported to the periplasm. This is also corrected here. 
r=model.reactions.get_by_id('CYOX')
model.metabolites.ficytC_p.charge=3
model.metabolites.focytC_p.charge=2
h_p=model.metabolites.h_p
ficytC_p =model.metabolites.ficytC_p
focytC_p=model.metabolites.focytC_p

r.add_metabolites({
    focytC_p:-8,
    ficytC_p: 8,
    h_c: -4.0,
    h_p: 4.0,
      })

print(r.check_mass_balance())
r


{'charge': 4.0}


Reaction identifier,CYOX
Name,R_CYOX
Memory address,0x020f45cc5fd0
Stoichiometry,"4.0 focytC_p + 8.0 h_c + o2_c <=> 4.0 ficytC_p + 2.0 h2o_c + 4.0 h_p 4.0 ferrocytochrome, cytc + 8.0 H + O2 <=> 4.0 ferricytochrome, cytc + 2.0 H2O + 4.0 H"
GPR,NGO1371 and NGO1373 and NGO1374 and (NGO1080 or NGO0101)
Lower bound,-1000.0
Upper bound,1000.0


In [68]:
r=model.reactions.get_by_id('CYOX')
model.reactions.CYOX.lower_bound=0
r

Reaction identifier,CYOX
Name,R_CYOX
Memory address,0x020f45cc5fd0
Stoichiometry,"4.0 focytC_p + 8.0 h_c + o2_c --> 4.0 ficytC_p + 2.0 h2o_c + 4.0 h_p 4.0 ferrocytochrome, cytc + 8.0 H + O2 --> 4.0 ferricytochrome, cytc + 2.0 H2O + 4.0 H"
GPR,NGO1371 and NGO1373 and NGO1374 and (NGO1080 or NGO0101)
Lower bound,0
Upper bound,1000.0


In [69]:
#Cytochrome oxidation was also flipped in the original UCYR reaction. Corrected here. 
#based on other examples from Bigg http://bigg.ucsd.edu/models/iAF987/reactions/CYTMQOR3pp
r=model.reactions.UCYR
r.add_metabolites({
    focytC_p:4,
    ficytC_p: -4,
      })

print(r.check_mass_balance())
r


{'charge': -2.0}


Reaction identifier,UCYR
Name,R_UCYR
Memory address,0x020f460aadc0
Stoichiometry,"2.0 ficytC_p + q8h2_c <=> 2.0 focytC_p + 2.0 h_p + q8_c 2.0 ferricytochrome, cytc + Ubiquinol-8 <=> 2.0 ferrocytochrome, cytc + 2.0 H + Ubiquinone-8"
GPR,NGO2031 and NGO2030 and NGO2029 and (NGO1080 or NGO0101 or Orphan)
Lower bound,-1000.0
Upper bound,1000.0


In [70]:
#Check Gc model ability to grow anaerobically on nitrite. 
def anaerobic_rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_no2_e_':
            reaction.lower_bound = -10. 

anaerobic_rpmi(model)
print(model.slim_optimize())
print(model.metabolites.no_p.summary())

0.0913683192189419
no_p
====
Formula: NO

Producing Reactions
-------------------
Percent  Flux Reaction                                                         Definition
 17.91%    10     NO2R             ficytC_p + h2o_p + no_p <=> focytC_p + 2.0 h_p + no2_p
 82.09% 45.84      NOR 2.0 ficytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 focytC_p + h2o_p + n2o_p

Consuming Reactions
-------------------
Percent   Flux Reaction    Definition
100.00% -55.84    NOtpp no_p <=> no_c


In [71]:
model.reactions.NOR

Reaction identifier,NOR
Name,R_NOR
Memory address,0x020f45f07850
Stoichiometry,"2.0 ficytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 focytC_p + h2o_p + n2o_p 2.0 ferricytochrome, cytc + 2.0 H + 2.0 Nitric-oxide <=> 2.0 ferrocytochrome, cytc + H2O + Nitrous-oxide"
GPR,NGO1275 and NGO1328
Lower bound,-1000.0
Upper bound,1000.0


In [72]:
#The original NOR was cytochrome oxidations were flipped. Corrected here. 

r=model.reactions.get_by_id('NOR')
r.name='nitrous-oxide:ferricytochrome-c oxidoreductase'
ficytC_p =model.metabolites.ficytC_p
focytC_p=model.metabolites.focytC_p
r.add_metabolites({
    focytC_p:-4,
    ficytC_p: 4,
      })

print(r.check_mass_balance())
r

{'charge': 2.0}


Reaction identifier,NOR
Name,nitrous-oxide:ferricytochrome-c oxidoreductase
Memory address,0x020f45f07850
Stoichiometry,"2.0 focytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 ficytC_p + h2o_p + n2o_p 2.0 ferrocytochrome, cytc + 2.0 H + 2.0 Nitric-oxide <=> 2.0 ferricytochrome, cytc + H2O + Nitrous-oxide"
GPR,NGO1275 and NGO1328
Lower bound,-1000.0
Upper bound,1000.0


In [73]:
#Model now grows anaerobically on nitrite. 
anaerobic_rpmi(model)
print(model.slim_optimize())
model.metabolites.no_p.summary()

0.06422017543764391


Percent,Flux,Reaction,Definition
100.00%,10,NO2R,ficytC_p + h2o_p + no_p <=> focytC_p + 2.0 h_p + no2_p
Percent,Flux,Reaction,Definition
100.00%,-10,NOR,2.0 focytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 ficytC_p + h2o_p + n2o_p


In [74]:
model.reactions.get_by_id('CYSabcpp')

Reaction identifier,CYSabcpp
Name,R_CYSabcpp
Memory address,0x020f45cc53a0
Stoichiometry,atp_c + cys_L_p + h2o_c --> adp_c + cys_L_c + h_c + pi_c ATP + L-Cysteine + H2O --> ADP + L-Cysteine + H + Phosphate
GPR,(NGO0374 and NGO0373 and NGO0372) or (NGO0968 and NGO0269 and NGO1152)
Lower bound,0.0
Upper bound,1000.0


In [75]:
#NGO2011, NGO2012, NGO2013, and NGO2014 are a cysteine (cys_L_) transporter. PMID: 22138345
#NGO0372, NGO373, NGO0374 are a cystine (cystine_L_) transporter. PMID: 22138345. In the current model, this is annotated as a cysteine importer 
#and no reaction for cystine import exists, or for breakdown of cystine. 
#Adding that transport and breakdown reaction here and correcting the gene annotations.

#NGO0968 and NGO0269 and NGO1152 is listed for lots of different transport reactions - its not very well annotated. Removing it from cystine and cysteine import. 

CYSabcpp = model.reactions.get_by_id('CYSabcpp')
CYSabcpp.gene_reaction_rule='NGO2011 and NGO2012 and NGO2013 and NGO2014'
CYSabcpp.notes = {'citations': 'PMID: 22138345', 'type': '', 'annotation': ''}


from cobra import Reaction, Metabolite

cystine_L_c = Metabolite(
    'cystine_L_c',
    formula='C6H12N2O4S2',
    name='Cystine',
    compartment='cytosol')

cystine_L_e = Metabolite(
    'cystine_L_e',
    formula='C6H12N2O4S2',
    name='Cystine',
    compartment='extracellular')

cystine_L_p = Metabolite(
    'cystine_L_p',
    formula='C6H12N2O4S2',
    name='Cystine',
    compartment='periplasm')


h_c = model.metabolites.get_by_id('h_c')
atp_c = model.metabolites.get_by_id('atp_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
adp_c = model.metabolites.get_by_id('adp_c')
pi_c = model.metabolites.get_by_id('pi_c')

CYSTINEabcpp = cobra.Reaction('CYSTINEabcpp')
CYSTINEabcpp.id = 'CYSTINEabcpp'
CYSTINEabcpp.name = 'cystine import'
CYSTINEabcpp.gene_reaction_rule = 'NGO0374 and NGO0373 and NGO0372'
CYSTINEabcpp.lower_bound = -1000.
CYSTINEabcpp.upper_bound = 1000.
CYSTINEabcpp.add_metabolites({
        atp_c: -1.0,
        cystine_L_p: -1.,
        h2o_c: -1. ,
        adp_c:  1.,
        cystine_L_c: 1. ,
        h_c: 1.,
        pi_c: 1.,
        })

model.add_reactions([CYSTINEabcpp])






EX_cystine_L_ = cobra.Reaction('EX_cystine_L_')
EX_cystine_L_.notes = {'citations': 'PMID: 22138345', 'type': '', 'annotation': ''}
EX_cystine_L_.name = 'exchange of cystine'
EX_cystine_L_.gene_reaction_rule = ''
EX_cystine_L_.lower_bound = 0.
EX_cystine_L_.upper_bound = 1000.
EX_cystine_L_.add_metabolites({
    cystine_L_e: -1.0,})


model.add_reactions([EX_cystine_L_])


h_c = model.metabolites.get_by_id('h_c')
nadh_c = model.metabolites.get_by_id('nadh_c')
nad_c = model.metabolites.get_by_id('nad_c')
cys_L_c = model.metabolites.get_by_id('cys_L_c')

CYSR = cobra.Reaction('CYSR')
CYSR.notes = {'citations': 'PMCID: PMC2323225', 'type': '', 'annotation': ''}
CYSR.name = 'cystine reductase'
CYSR.gene_reaction_rule = 'Orphan'
CYSR.lower_bound = -1000.
CYSR.upper_bound = 1000.
CYSR.add_metabolites({
    cys_L_c: -2.0,
    nad_c: -1.0, 
    cystine_L_c: 1.0,
    h_c: 1.0,
    nadh_c: 1.0,
    })


model.add_reactions([CYSR])
print(CYSR.check_mass_balance())





{}


In [76]:
#Neisseria, like many other proteobacteria, lack the permease portion of the phosphotransferase. 
#And therefore do not use it for carbohydrate transport.
#The remaining portions of the system are thought to perform regulatory roles, related to virulence. 
#PMID: 26858137

model.remove_reactions(model.reactions.GLCptspp)
model.remove_reactions(model.reactions.FRUpts2pp)
model.remove_reactions(model.reactions.ASCBptspp)
model.remove_reactions(model.reactions.MALTptspp)
model.remove_reactions(model.reactions.MANptspp)
model.remove_reactions(model.reactions.SUCptspp)

C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [77]:
#NADPH does not catalyze the FabI reaction, only NADH. Therefore all of the NADPH dependent reactions were removed. 
#PMID: 26567338   
model.remove_reactions(model.reactions.EAR120y)
model.remove_reactions(model.reactions.EAR100y)
model.remove_reactions(model.reactions.EAR121y)
model.remove_reactions(model.reactions.EAR140y)
model.remove_reactions(model.reactions.EAR141y)
model.remove_reactions(model.reactions.EAR160y)
model.remove_reactions(model.reactions.EAR161y)
model.remove_reactions(model.reactions.EAR180y)
model.remove_reactions(model.reactions.EAR181y)
model.remove_reactions(model.reactions.EAR40y)
model.remove_reactions(model.reactions.EAR60y)
model.remove_reactions(model.reactions.EAR80y)

    

In [78]:
#Gc can only grow on glucose, lactate, and pyruvate. It cannot grow on gluconate. 
#Despite similarities to gluconate transporters and gluconate kinase, Nm also can't grow on gluconate so this should be removed. 
#PMID: 27454890
model.remove_reactions(model.reactions.GLCNtex)
model.remove_reactions(model.reactions.EX_glcn_e_)


In [79]:
model

Name,AE004969.1
Memory address,0x020f3d096130
Number of metabolites,1337
Number of reactions,1504
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"Cytosol, C_c, C_p, c, periplasm, cytosol, extracellular"


In [80]:
for metabolite in model.metabolites:
    if '_c' in metabolite.id:
        metabolite.compartment = 'cytosol'
    if '_p' in metabolite.id:
        metabolite.compartment = 'periplasm'
    if '_e' in metabolite.id:
        metabolite.compartment = 'extracellular'

In [81]:
cobra.io.save_json_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/test.json')

## 2b. Correct Reaction Mass/Charge Balances

In [82]:
#After adding genes and associated reactions from CarveMe FA1090 model, ensure all reactions are charge and mass balanced.
#All formulas and charges are pulled from bigg unless indicated


In [83]:
#Define biomass reactions to exclude from imbalance check
biomass_reactions = set(['PLIPIDS', 'PLIPIDS_ESS', 'PROTS', 'PEPGLY','RNAS', 'DNAS' ])


#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

158 158 158


,Reactions,Charge,Masses
0,3HAD161x,,"{'H': 96.0, 'O': 17.0, 'Ab': 1.0, 'A': 2.0, 'C..."
1,3OAR161x,,"{'C': -27.0, 'H': -49.0, 'N': -2.0, 'O': -9.0,..."
2,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
3,ACCOAC,,"{'C': -9.999999999621423e-06, 'H': -1.49999999..."
4,ACHBS,,"{'C': -1.2e-06, 'H': -1.6e-06, 'O': -7e-07, 'N..."
...,...,...,...
153,PTHPS,,"{'C': -9.0, 'H': -11.0, 'N': -5.0, 'O': -3.0}"
154,CPH4S,,"{'C': -7.0, 'H': -8.0, 'O': -3.0, 'N': -5.0}"
155,CDGS,,"{'H': 3.0, 'N': 1.0}"
156,CDGR,,{'H': -5.0}


In [84]:
#Model does not have charges associated with metabolites. Adding charges and formulas.
#load metabolite annotation table
metabolites= list()
for mets in model.metabolites:
    metabolites.append(mets.id)
metabolites

#Use Mendum 2011 metabolite formulas
annotation_table = pd.read_csv (cwd+'/Gc_GENRE_2022/Curate_Gc_Model/Annotation_resources/Metabolite_charges.csv', usecols= ['Name','formula', 'charge'],index_col=0)
annotation_table = annotation_table.fillna('')
annotation_table                       


,formula,charge
Name,,
10fthf_c,C20H21N7O7,-2
12dgr120_c,C27H52O5,0
12dgr120_p,C27H52O5,0
12dgr140_c,C31H60O5,0
12dgr140_p,C31H60O5,0
...,...,...
xylu_L_e,C5H10O5,0
xylu_L_p,C5H10O5,0
zn2_c,Zn,2


In [85]:
cobra.io.save_json_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/test.json')

In [86]:
#Add metabolite charges to model from annotation table
for x in model.metabolites:
    metid=x.id
    if metid in annotation_table.index:
        charge=annotation_table['charge'].loc[metid]
        model.metabolites.get_by_id(metid).charge= int(charge)
        formula=annotation_table['formula'].loc[metid]
        model.metabolites.get_by_id(metid).formula= str(formula)
    else:
        pass

In [87]:
cobra.io.save_json_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/test.json')

In [88]:
#Define biomass reactions to exclude from imbalance check
biomass_reactions = set(['PLIPIDS', 'PLIPIDS_ESS', 'PROTS', 'PEPGLY','RNAS', 'DNAS' ])


#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

164 164 164


,Reactions,Charge,Masses
0,3OAS161a,1.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ACCOAC,0.000001,"{'C': -9.999999999621423e-06, 'H': -1.49999999..."
2,ACHBS,0.0,"{'C': -1.2e-06, 'H': -1.6e-06, 'O': -7e-07, 'N..."
3,ACLS,0.0,"{'C': -1.2e-06, 'H': -1.6e-06, 'O': -7e-07, 'N..."
4,ACPH,-1.0,"{'H': -2.0, 'O': -1.0, 'R': -1.0}"
...,...,...,...
159,UAPGR,0.0,"{'C': -2.69999999957804e-06, 'H': -3.100000000..."
160,UPPRT,-1.0,{'H': -1.0}
161,XPPT,-1.0,{'H': -1.0}
162,XPRT,-1.0,{'H': -1.0}


### Remove Cofactors

In [89]:
#Remove biotin as a cofactor
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "btn_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  btn_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('btn_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('btn_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.btn_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.btn_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

   ACCOAC :  btn_c : -1e-06
     BTS4 :  btn_c : 1.0


In [90]:
#Biotin has been removed as a cofactor all reactions
#example
model.reactions.ACCOAC

Reaction identifier,ACCOAC
Name,R_ACCOAC
Memory address,0x020f45c0c850
Stoichiometry,accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c Acetyl-CoA + ATP + Bicarbonate --> ADP + H + Malonyl-CoA + Phosphate
GPR,(NGO0821 or NGO0821) and NGO0044 and NGO0249 and NGO0045 and NGO2001
Lower bound,0.0
Upper bound,1000.0


In [91]:
#But Biotin has not been removed if it is an actual reactant
#example
model.reactions.BTS4

Reaction identifier,BTS4
Name,R_BTS4
Memory address,0x020f45c9ef10
Stoichiometry,amet_c + dtbt_c + h_c --> btn_c + dad_5_c + h2o_c + met_L_c + o2_c S-Adenosyl-L-methionine + Dethiobiotin + H --> Biotin + 5-Deoxyadenosine + H2O + L-Methionine + O2
GPR,NGO0813 or NGO0813
Lower bound,0.0
Upper bound,1000.0


In [92]:
model.slim_optimize()

-1.0917825069055096e-19

In [93]:
#Remove thmpp_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "thmpp_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  thmpp_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('thmpp_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('thmpp_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.thmpp_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.thmpp_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

    ACHBS :  thmpp_c : -1e-07
     ACLS :  thmpp_c : -1e-07
       CS :  thmpp_c : -1e-06
     DXPS :  thmpp_c : -1e-07
      PDH :  thmpp_c : -1e-06


In [94]:
#example
model.reactions.ACHBS

Reaction identifier,ACHBS
Name,R_ACHBS
Memory address,0x020f45c10970
Stoichiometry,2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c 2-Oxobutanoate + H + Pyruvate --> S-2-Aceto-2-hydroxybutanoate + CO2
GPR,NGO1236 and (Blank or NGO1235)
Lower bound,0.0
Upper bound,1000.0


In [95]:
#Remove fad_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "fad_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  fad_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('fad_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('fad_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.fad_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.fad_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

    ASPO3 :  fad_c : -1e-07
    ASPO5 :  fad_c : -1e-07
    ASPO6 :  fad_c : -1e-07
    CHORS :  fad_c : -1e-07
   DHORD2 :  fad_c : -1e-07
     FUMR :  fad_c : -1e-08
   MTHFR2 :  fad_c : -1e-07
 NADH16pp :  fad_c : -1e-07
    NADH5 :  fad_c : -1e-07
  NADHQNa :  fad_c : -1e-07
 NADPHQR2 :  fad_c : -1e-07
    PPPGO :  fad_c : -1e-07
    SUCD1 :  fad_c : -1e-08
    UAPGR :  fad_c : -1e-07


In [96]:
#Remove pydx5p_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "pydx5p_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  pydx5p_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('pydx5p_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('pydx5p_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.pydx5p_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.pydx5p_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

     ADCL :  pydx5p_c : -1e-07
     ADCS :  pydx5p_c : -1e-07
   AMAOTr :  pydx5p_c : -1e-07
    ARGDC :  pydx5p_c : -1e-07
     CYSS :  pydx5p_c : -1e-07
    DAPDC :  pydx5p_c : -1e-07
     PGCD :  pydx5p_c : -1e-07
   PSD120 :  pydx5p_c : -1e-07
   PSD140 :  pydx5p_c : -1e-07
   PSD141 :  pydx5p_c : -1e-07
  PSD141b :  pydx5p_c : -1e-07
   PSD160 :  pydx5p_c : -1e-07
  PSD160a :  pydx5p_c : -1e-07
  PSD160b :  pydx5p_c : -1e-07
  PSD160c :  pydx5p_c : -1e-07
  PSD160e :  pydx5p_c : -1e-07
  PSD161a :  pydx5p_c : -1e-07
   PSD180 :  pydx5p_c : -1e-07
   PSD181 :  pydx5p_c : -1e-07
    PSERT :  pydx5p_c : -1e-07
   SERD_L :  pydx5p_c : -1e-07
    SHSL5 :  pydx5p_c : -1e-07
   THRD_L :  pydx5p_c : -1e-07
     THRS :  pydx5p_c : -1e-07


In [97]:
#Remove nad_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "nad_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  nad_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('nad_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('nad_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.nad_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.nad_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

    AGMHE :  nad_c : -1e-07
   AMPMS2 :  nad_c : -1.0
     DHQS :  nad_c : -1e-07
 GLYCLTDx :  nad_c : 1.0
     IPPS :  nad_c : -1e-07
   MTHFR2 :  nad_c : 1.0
  NADHQNa :  nad_c : 1.0


In [98]:
#Remove zn2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "zn2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  zn2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('zn2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('zn2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.zn2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.zn2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

AMPTASECG :  zn2_c : -1e-07
     METS :  zn2_c : -1e-07
   PPBNGS :  zn2_c : -1e-07


In [99]:
#Remove ca2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "ca2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  ca2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('ca2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('ca2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.ca2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.ca2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

PLIPA1A120pp :  ca2_c : -1e-07
PLIPA1A140pp :  ca2_c : -1e-07
PLIPA1A141b :  ca2_c : -1e-07
PLIPA1A141pp :  ca2_c : -1e-07
PLIPA1A160a :  ca2_c : -1e-07
PLIPA1A160b :  ca2_c : -1e-07
PLIPA1A160c :  ca2_c : -1e-07
PLIPA1A160e :  ca2_c : -1e-07
PLIPA1A160pp :  ca2_c : -1e-07
PLIPA1A161a :  ca2_c : -1e-07
PLIPA1A180pp :  ca2_c : -1e-07
PLIPA1A181pp :  ca2_c : -1e-07
PLIPA1E120pp :  ca2_c : -1e-07
PLIPA1E140pp :  ca2_c : -1e-07
PLIPA1E141b :  ca2_c : -1e-07
PLIPA1E141pp :  ca2_c : -1e-07
PLIPA1E160a :  ca2_c : -1e-07
PLIPA1E160b :  ca2_c : -1e-07
PLIPA1E160c :  ca2_c : -1e-07
PLIPA1E160e :  ca2_c : -1e-07
PLIPA1E160pp :  ca2_c : -1e-07
PLIPA1E161a :  ca2_c : -1e-07
PLIPA1E180pp :  ca2_c : -1e-07
PLIPA1E181pp :  ca2_c : -1e-07
PLIPA1G120pp :  ca2_c : -1e-07
PLIPA1G140pp :  ca2_c : -1e-07
PLIPA1G141b :  ca2_c : -1e-07
PLIPA1G141pp :  ca2_c : -1e-07
PLIPA1G160a :  ca2_c : -1e-07
PLIPA1G160b :  ca2_c : -1e-07
PLIPA1G160c :  ca2_c : -1e-07
PLIPA1G160e :  ca2_c : -1e-07
PLIPA1G160pp :  ca2_c : -

In [100]:
#Remove mn2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "mn2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  mn2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('mn2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('mn2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.mn2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.mn2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

  SPODMMn :  mn2_c : -1e-07


In [101]:
#Remove fe2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "fe2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  fe2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('fe2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('fe2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.fe2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.fe2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

PHEMEFErel :  fe2_c : 1.0
    PROHX :  fe2_c : -1e-07


In [102]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

104 104 104


,Reactions,Charge,Masses
0,3OAS161a,1.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ACPH,-1.0,"{'H': -2.0, 'O': -1.0, 'R': -1.0}"
2,ACPS1,,"{'H': 1.0, 'O': 1.0, 'R': 1.0}"
3,ADPT,-1.0,{'H': -1.0}
4,AGPAT141b,2.0,{}
...,...,...,...
99,TSULST,-2.0,{'H': -2.0}
100,UPPRT,-1.0,{'H': -1.0}
101,XPPT,-1.0,{'H': -1.0}
102,XPRT,-1.0,{'H': -1.0}


### Additional Mass/Charge Curation

In [103]:
#Correct Imbalances for ACP reactions
#ACPH
r = model.reactions.get_by_id('ACPH')
print(r.check_mass_balance())
r


{'charge': -1.0, 'H': -2.0, 'O': -1.0, 'R': -1.0}


Reaction identifier,ACPH
Name,R_ACPH
Memory address,0x020f45c215b0
Stoichiometry,ACP_c + h2o_c --> pan4p_c acyl-carrier-protein + H2O --> Pantetheine-4-phosphate
GPR,Orphan
Lower bound,0.0
Upper bound,1000.0


In [104]:
apoACP_c = Metabolite(
    'apoACP_c',
    formula='HOR',
    name='apo acyl-carrier-protein',
    compartment='cytosol')

h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    apoACP_c: 1.0,
    h_c: 1.0})

print(r)
print(r.check_mass_balance())

ACPH: ACP_c + h2o_c --> apoACP_c + h_c + pan4p_c
{}


In [105]:
model.slim_optimize()

0.0

In [106]:
#ACPS1
r = model.reactions.get_by_id('ACPS1')
print(r.check_mass_balance())
r


{'H': 1.0, 'O': 1.0, 'R': 1.0}


Reaction identifier,ACPS1
Name,R_ACPS1
Memory address,0x020f45c21850
Stoichiometry,coa_c --> ACP_c + h_c + pap_c Coenzyme-A --> acyl-carrier-protein + H + Adenosine-3-5-bisphosphate
GPR,NGO1507
Lower bound,0.0
Upper bound,1000.0


In [107]:
r.add_metabolites({
    apoACP_c: -1.0})

print(r)
print(r.check_mass_balance())

ACPS1: apoACP_c + coa_c --> ACP_c + h_c + pap_c
{}


In [108]:
#Add apoACP import reaction. This is required to show "synthesis" of a protein based cofactor. 
#Integration of Biomass Formulations of Genome-Scale Metabolic Models with Experimental Data Reveals Universally Essential Cofactors in Prokaryotes
#Joana C Xavier, Kiran Raosaheb Patil, Isabel Rocha
#PMID: 27939572 PMCID: PMC5249239

apoACP_c = model.metabolites.get_by_id('apoACP_c')

EX_apoACP_c_ = cobra.Reaction('EX_apoACP_c_')
EX_apoACP_c_.notes = {'citations': 'PMCID: PMC5249239', 'type': 'essential cofactor', 'annotation': ''}
EX_apoACP_c_.name = 'exchange of apoACP'
EX_apoACP_c_.gene_reaction_rule = ''
EX_apoACP_c_.lower_bound = 0.
EX_apoACP_c_.upper_bound = 1000.
EX_apoACP_c_.add_metabolites({
    apoACP_c: -1.0,
  
})

model.add_reactions([EX_apoACP_c_])

In [109]:
r=model.reactions.get_by_id('EX_apoACP_c_')
r

Reaction identifier,EX_apoACP_c_
Name,exchange of apoACP
Memory address,0x020f4d19ce80
Stoichiometry,apoACP_c --> apo acyl-carrier-protein -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [110]:
complete(model)
model.slim_optimize()

2.234503239508173

In [111]:
#AMPMS2
r = model.reactions.get_by_id('AMPMS2')
print(r.check_mass_balance())
r

{'charge': 2.0, 'H': 1.0, 'O': -3.0, 'P': -1.0}


Reaction identifier,AMPMS2
Name,R_AMPMS2
Memory address,0x020f45c699a0
Stoichiometry,air_c + h2o_c + nad_c --> 4ahmmp_c + 2.0 for_c + 3.0 h_c + nadh_c 5-amino-1-5-phospho-D-ribosyl-imidazole + H2O + Nicotinamide-adenine-dinucleotide --> 4-Amino-5-hydroxymethyl-2-methylpyrimidine + 2.0 Formate + 3.0 H + Nicotinamide-adenine-dinucleotide--reduced
GPR,NGO2041
Lower bound,0.0
Upper bound,1000.0


In [112]:
#The reaction AMPMS2 should use 4ampm rather than 4ahmmp according to BIGG and KEGG. Replaced this metabolite in the reaction.


In [113]:
met1=model.metabolites.get_by_id('4ampm_c')
met2 = model.metabolites.get_by_id('4ahmmp_c')

r.subtract_metabolites({
    met2: 1.0})
r.add_metabolites({
    met1: 1.0})

print(r.check_mass_balance())
r

{}


Reaction identifier,AMPMS2
Name,R_AMPMS2
Memory address,0x020f45c699a0
Stoichiometry,air_c + h2o_c + nad_c --> 4ampm_c + 2.0 for_c + 3.0 h_c + nadh_c 5-amino-1-5-phospho-D-ribosyl-imidazole + H2O + Nicotinamide-adenine-dinucleotide --> 4-Amino-2-methyl-5-phosphomethylpyrimidine + 2.0 Formate + 3.0 H + Nicotinamide-adenine-dinucleotide--reduced
GPR,NGO2041
Lower bound,0.0
Upper bound,1000.0


In [114]:
#However, now model doesn't grow in RPMI which doesn't contain added thiamine. Thiamine synthesis seems to have been disrupted by changing the metabolite. 
model.slim_optimize()

2.234503239386048

In [115]:
#ASNSYN
r = model.reactions.get_by_id('ASNSYN')
print(r.check_mass_balance())
r

{'charge': -2.0, 'H': 2.0, 'O': 2.0}


Reaction identifier,ASNSYN
Name,R_ASNSYN
Memory address,0x020f45c81f70
Stoichiometry,asp_L_c + atp_c + gln_L_c + h_c --> adp_c + asn_L_c + glu_L_c + h2o_c + pi_c L-Aspartate + ATP + L-Glutamine + H --> ADP + L-Asparagine + L-Glutamate + H2O + Phosphate
GPR,NGO1489 and NGO0663 and NGO0662 and NGO0660
Lower bound,0.0
Upper bound,1000.0


In [116]:
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')

r.add_metabolites({
    h2o_c: -2.0,
    h_c: 2.0})

print(r)
print(r.check_mass_balance())

ASNSYN: asp_L_c + atp_c + gln_L_c + h2o_c --> adp_c + asn_L_c + glu_L_c + h_c + pi_c
{}


In [117]:
#BTS4
r = model.reactions.get_by_id('BTS4')
print(r.check_mass_balance())
r

{'charge': -2.0, 'O': 3.0, 'S': 1.0}


Reaction identifier,BTS4
Name,R_BTS4
Memory address,0x020f45c9ef10
Stoichiometry,amet_c + dtbt_c + h_c --> btn_c + dad_5_c + h2o_c + met_L_c + o2_c S-Adenosyl-L-methionine + Dethiobiotin + H --> Biotin + 5-Deoxyadenosine + H2O + L-Methionine + O2
GPR,NGO0813 or NGO0813
Lower bound,0.0
Upper bound,1000.0


In [118]:
#Reaction will remain imbalanced for Sulfer. The sulfer is donated by the enzyme itself and not by a metabolite.
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
o2_c = model.metabolites.get_by_id('o2_c')

r.add_metabolites({
    h2o_c: -1.0,
    h_c: 2.0,
    o2_c: -1.0
})

print(r)
print(r.check_mass_balance())

BTS4: amet_c + dtbt_c --> btn_c + dad_5_c + h_c + met_L_c
{'S': 1.0}


In [119]:
#CTPS3
r = model.reactions.get_by_id('CTPS3')
print(r.check_mass_balance())
r

{'charge': 1.0, 'H': 1.0}


Reaction identifier,CTPS3
Name,R_CTPS3
Memory address,0x020f45cc1b50
Stoichiometry,ctp_c + h2o_c <=> nh4_c + utp_c CTP + H2O <=> Ammonium + UTP
GPR,NGO0421
Lower bound,-1000.0
Upper bound,1000.0


In [120]:
r.add_metabolites({
    h_c:-1
})

print(r)
print(r.check_mass_balance())

CTPS3: ctp_c + h2o_c + h_c <=> nh4_c + utp_c
{}


In [121]:
#DHORD2
r = model.reactions.get_by_id('DHORD2')
print(r.check_mass_balance())
r

{'charge': 1.9999999989472883e-07, 'C': -1.6999999985500835e-06, 'H': -1.900000000887303e-06, 'N': -3.9999999978945766e-07, 'O': -8.999999998593466e-07, 'P': -1e-07}


Reaction identifier,DHORD2
Name,R_DHORD2
Memory address,0x020f45cff760
Stoichiometry,dhor_S_c + 1e-07 fmn_c + q8_c --> orot_c + q8h2_c S-Dihydroorotate + 1e-07 FMN + Ubiquinone-8 --> Orotate + Ubiquinol-8
GPR,NGO1761
Lower bound,0.0
Upper bound,1000.0


In [122]:
fmn_c = model.metabolites.get_by_id('fmn_c')

r.add_metabolites({
    fmn_c: 1e-07,
})

print(r)
print(r.check_mass_balance())

DHORD2: dhor_S_c + q8_c --> orot_c + q8h2_c
{}


In [123]:
#DHQS
r = model.reactions.get_by_id('DHQS')
print(r.check_mass_balance())
r

{'charge': -1.9999999989472883e-07, 'Co': -1e-07}


Reaction identifier,DHQS
Name,R_DHQS
Memory address,0x020f45cfff70
Stoichiometry,2dda7p_c + 1e-07 cobalt2_c --> 3dhq_c + pi_c 2-Dehydro-3-deoxy-D-arabino-heptonate-7-phosphate + 1e-07 Co2 --> 3-Dehydroquinate + Phosphate
GPR,NGO0092
Lower bound,0.0
Upper bound,1000.0


In [124]:
cobalt2_c = model.metabolites.get_by_id('cobalt2_c')

r.add_metabolites({
    cobalt2_c: 1e-07,
})

print(r)
print(r.check_mass_balance())

DHQS: 2dda7p_c --> 3dhq_c + pi_c
{}


In [125]:
model.slim_optimize()

2.2345032345098805

In [126]:
#GLYCLTDx
r = model.reactions.get_by_id('GLYCLTDx')
print(r.check_mass_balance())
r

{'charge': 1.0, 'H': 1.0}


Reaction identifier,GLYCLTDx
Name,R_GLYCLTDx
Memory address,0x020f45e02970
Stoichiometry,glx_c + nadh_c --> glyclt_c + nad_c Glyoxylate + Nicotinamide-adenine-dinucleotide--reduced --> Glycolate + Nicotinamide-adenine-dinucleotide
GPR,NGO2043 or NGO1243
Lower bound,0.0
Upper bound,1000.0


In [127]:
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: -1,
})

print(r)
print(r.check_mass_balance())

GLYCLTDx: glx_c + h_c + nadh_c --> glyclt_c + nad_c
{}


In [128]:
#GLYCLTDy
r = model.reactions.get_by_id('GLYCLTDy')
print(r.check_mass_balance())
r

{'charge': 1.0, 'H': 1.0}


Reaction identifier,GLYCLTDy
Name,R_GLYCLTDy
Memory address,0x020f45e022e0
Stoichiometry,glx_c + nadph_c --> glyclt_c + nadp_c Glyoxylate + Nicotinamide-adenine-dinucleotide-phosphate--reduced --> Glycolate + Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO2043
Lower bound,0.0
Upper bound,1000.0


In [129]:
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: -1,
})

print(r)
print(r.check_mass_balance())

GLYCLTDy: glx_c + h_c + nadph_c --> glyclt_c + nadp_c
{}


In [130]:
#H2AKGAL
r = model.reactions.get_by_id('H2AKGAL')
print(r.check_mass_balance())
r

{'charge': 1.0, 'H': 1.0}


Reaction identifier,H2AKGAL
Name,R_H2AKGAL
Memory address,0x020f45e288e0
Stoichiometry,4h2oglt_c --> glx_c + h_c + pyr_c D-4-Hydroxy-2-oxoglutarate --> Glyoxylate + H + Pyruvate
GPR,NGO0713
Lower bound,0.0
Upper bound,1000.0


In [131]:
#H2AKGAL is not consistent with the bigg name for this same reaction. Corrected name to DDPGA. 
#Similarly, the protenation of 4h2oglt_c was used from kegg, which is incorrect for neutral ph.
#corrected to the formula from bigg. And removed the now unnecessary hydrogens from the reaction.
r.id  ='DDPGA'
r.name ='R_DDPGA'
model.metabolites.get_by_id('4h2oglt_c').formula = 'C5H4O6'
h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1.0,
})

print(r)
print(r.check_mass_balance())

DDPGA: 4h2oglt_c --> glx_c + 2.0 h_c + pyr_c
{'charge': 2.0, 'H': 2.0}


In [132]:
#HGTA
r = model.reactions.get_by_id('HGTA')
print(r.check_mass_balance())
r

{'charge': 1.0, 'H': 1.0}


Reaction identifier,HGTA
Name,R_HGTA
Memory address,0x020f45e34a90
Stoichiometry,akg_c + e4hglu_c <=> 4h2oglt_c + glu_L_c + h_c 2-Oxoglutarate + L-erythro-4-Hydroxyglutamate <=> D-4-Hydroxy-2-oxoglutarate + L-Glutamate + H
GPR,NGO1452
Lower bound,-1000.0
Upper bound,1000.0


In [133]:
#HGTA is not consistent with the bigg name for this same reaction. Corrected name to EHGLAT. 
#Similarly, the protenation of e4hglu_c was used from kegg, which is incorrect for neutral ph.
#corrected to the formula from bigg. And removed the now unnecessary hydrogens from the reaction.
r.id  ='EHGLAT'
r.name ='R_EHGLAT'
model.metabolites.get_by_id('e4hglu_c').formula = 'C5H8NO5'
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: -1.0,
})

print(r)
print(r.check_mass_balance())

EHGLAT: akg_c + e4hglu_c <=> 4h2oglt_c + glu_L_c
{}


In [134]:
model.slim_optimize()

2.2345032345098805

In [135]:
#HMT
r = model.reactions.get_by_id('HMT')
print(r.check_mass_balance())
r

{'charge': -1.0, 'H': -1.0}


Reaction identifier,HMT
Name,R_HMT
Memory address,0x020f45e41880
Stoichiometry,amet_c + his_L_c --> ahcys_c + mhis_c S-Adenosyl-L-methionine + L-Histidine --> S-Adenosyl-L-homocysteine + N(pi)-Methyl-L-histidine
GPR,NGO0746
Lower bound,0.0
Upper bound,1000.0


In [136]:
model.remove_reactions('r')

C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")
C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:738: UserWarning: r not in AE004969.1
  warn("%s not in %s" % (reaction, self))


In [137]:
#MCITS
r = model.reactions.get_by_id('MCITS')
print(r.check_mass_balance())
r

{'charge': -3.0, 'H': -3.0}


Reaction identifier,MCITS
Name,R_MCITS
Memory address,0x020f45e98a30
Stoichiometry,h2o_c + 2.0 h_c + oaa_c + ppcoa_c --> 2mcit_c + coa_c H2O + 2.0 H + Oxaloacetate + Propanoyl-CoA --> 2-Methylcitrate + Coenzyme-A
GPR,NGO1525
Lower bound,0.0
Upper bound,1000.0


In [138]:
h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 3.0,
})

print(r)
print(r.check_mass_balance())

MCITS: h2o_c + oaa_c + ppcoa_c --> 2mcit_c + coa_c + h_c
{}


In [139]:
#MTHF
r = model.reactions.get_by_id('MTHF')
print(r.check_mass_balance())
r

{'H': -2.0, 'O': 1.0}


Reaction identifier,MTHF
Name,R_MTHF
Memory address,0x020f45ecbc70
Stoichiometry,5mthf_c --> 10fthf_c 5-Methyltetrahydrofolate --> 10-Formyltetrahydrofolate
GPR,Sgene
Lower bound,0.0
Upper bound,1000.0


In [140]:
##MTHF is a spontaneous reaction. Uncertain how frequently this occurs in nature. 
#Typically found in carbon fixation pathways in KEGG and in multistep, requiring NADH. https://www.kegg.jp/pathway/map00720+C00234
#Added NADH here for mass and charge balance. 
#Corrected h2o and hydrogens.
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
nad_c = model.metabolites.get_by_id('nad_c')
nadh_c=model.metabolites.get_by_id('nadh_c')

model.metabolites.get_by_id('5mthf_c').charge=-2
model.metabolites.get_by_id('5mthf_c').formula= 'C20H23N7O6'

r.add_metabolites({
    h2o_c: -1.0,
    h_c: 2.0,
    nad_c: -2.0,
    nadh_c: 2.0,
})

print(r)
print(r.check_mass_balance())

#Charge balance is off

MTHF: 5mthf_c + h2o_c + 2.0 nad_c --> 10fthf_c + 2.0 h_c + 2.0 nadh_c
{}


In [141]:
#NADHQNa
r = model.reactions.get_by_id('NADHQNa')
print(r.check_mass_balance())
r

{'charge': 2.0000000011677344e-07, 'C': -1.6999999985500835e-06, 'H': -1.900000000887303e-06, 'N': -4.0000000023354687e-07, 'O': -8.999999998593466e-07, 'P': -1.0000000005838672e-07}


Reaction identifier,NADHQNa
Name,R_NADHQNa
Memory address,0x020f45ed6820
Stoichiometry,1e-07 fmn_c + h_c + 2.0 na1_c + nadh_c + q8_c --> 2.0 na1_p + nad_c + q8h2_c 1e-07 FMN + H + 2.0 Sodium + Nicotinamide-adenine-dinucleotide--reduced + Ubiquinone-8 --> 2.0 Sodium + Nicotinamide-adenine-dinucleotide + Ubiquinol-8
GPR,NGO1418 and NGO1417 and NGO1416 and NGO1415 and NGO1414 and NGO1413
Lower bound,0.0
Upper bound,1000.0


In [142]:
fmn_c = model.metabolites.get_by_id('fmn_c')


r.add_metabolites({
    fmn_c: 1e-07,
})

print(r)
print(r.check_mass_balance())

NADHQNa: h_c + 2.0 na1_c + nadh_c + q8_c --> 2.0 na1_p + nad_c + q8h2_c
{}


In [143]:
#Imbalance progress
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

91 91 91


,Reactions,Charge,Masses
0,3OAS161a,1.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ADPT,-1.0,{'H': -1.0}
2,AGPAT141b,2.0,{}
3,AGPAT160b,2.0,{}
4,AGPAT160c,2.0,{}
...,...,...,...
86,TSULST,-2.0,{'H': -2.0}
87,UPPRT,-1.0,{'H': -1.0}
88,XPPT,-1.0,{'H': -1.0}
89,XPRT,-1.0,{'H': -1.0}


In [144]:
model.slim_optimize()

2.2345042552092766

In [145]:
r= model.reactions.get_by_id('PLIPIDS')
print(r)

PLIPIDS: 0.001857 pa120_p + 0.001157 pa140_p + 0.001202 pa141_p + 0.003838 pa160_p + 0.006675 pa161_p + 0.000484 pa180_p + 0.0008971 pa181_p + 0.004466 pe120_p + 0.010539 pe140_p + 0.002913 pe141_p + 0.004721 pe141b_p + 0.009371 pe160_p + 0.01007 pe160a_p + 0.02591 pe160c_p + 0.0186 pe160e_p + 0.01562 pe161_p + 0.001189 pe180_p + 0.022 pe181_p + 0.001551 pg120_p + 0.001283 pg140_p + 0.001016 pg141_p + 0.001668 pg141b_p + 0.003285 pg160_p + 0.002712 pg160a_p + 0.001644 pg160b_p + 0.006129 pg160c_p + 0.006099 pg161_p + 0.000418 pg180_p + 0.000774 pg181_p <=> 0.1483 lipid_NM_p


In [146]:
#Phospholipids adjusted for Gc specific values based on "Biomass composition from nmb_itm560 updated for Gc" spreadsheet
r= model.reactions.get_by_id('PLIPIDS')

pe141b_p= model.metabolites.get_by_id('pe141b_p')
pe160a_p= model.metabolites.get_by_id('pe160a_p')
pe160c_p= model.metabolites.get_by_id('pe160c_p')
pe160e_p= model.metabolites.get_by_id('pe160e_p')
pg141b_p= model.metabolites.get_by_id('pg141b_p')
pg160a_p= model.metabolites.get_by_id('pg160a_p')
pg160b_p= model.metabolites.get_by_id('pg160b_p')
pg160c_p= model.metabolites.get_by_id('pg160c_p')

pa120_p= model.metabolites.get_by_id('pa120_p')
pa140_p= model.metabolites.get_by_id('pa140_p')
pa141_p= model.metabolites.get_by_id('pa141_p')
pa160_p= model.metabolites.get_by_id('pa160_p')
pa161_p= model.metabolites.get_by_id('pa161_p')
pa180_p= model.metabolites.get_by_id('pa180_p')
pa181_p= model.metabolites.get_by_id('pa181_p')

pe120_p= model.metabolites.get_by_id('pe120_p')
pe140_p= model.metabolites.get_by_id('pe140_p')
pe141_p= model.metabolites.get_by_id('pe141_p')
pe160_p= model.metabolites.get_by_id('pe160_p')
pe161_p= model.metabolites.get_by_id('pe161_p')
pe180_p= model.metabolites.get_by_id('pe180_p')
pe181_p= model.metabolites.get_by_id('pe181_p')

pg120_p= model.metabolites.get_by_id('pg120_p')
pg140_p= model.metabolites.get_by_id('pg140_p')
pg141_p= model.metabolites.get_by_id('pg141_p')
pg160_p= model.metabolites.get_by_id('pg160_p')
pg161_p= model.metabolites.get_by_id('pg161_p')
pg180_p= model.metabolites.get_by_id('pg180_p')
pg181_p= model.metabolites.get_by_id('pg181_p')


r.add_metabolites({
 
    pa120_p:  0.001857, 
    pa140_p: 0.001157 ,
    pa160_p: 0.003838,
    pa180_p:  0.000484,
    pa141_p: 0.001202 ,
    pa161_p: 0.006675,
    pa181_p: 0.0008971, 
    pe120_p: 0.004466 ,
    pe140_p: 0.010539 ,
    pe141b_p: 0.004721, 
    pe160_p: 0.009371 ,
    pe160a_p: 0.01007 ,
    pe160c_p: 0.02591 ,
    pe160e_p: 0.0186,
    pe180_p: 0.001189, 
    pe141_p: 0.002913 ,
    pe161_p: 0.01562 ,
    pe181_p:  0.022 ,
    pg120_p: 0.001551 ,
    pg140_p: 0.001283 ,
    pg160_p: 0.003285 ,
    pg160a_p:  0.002712, 
    pg160b_p: 0.001644 ,
    pg160c_p:  0.006129, 
    pg161_p: 0.006099 ,
    pg181_p: 0.000774 ,
    pg180_p: 0.000418 ,
    pg141_p: 0.001016 ,
    pg141b_p: 0.001668, })

  
r.add_metabolites({   
    pa120_p: -0.000503555042155509,
    pa160_p: -0.00207597350252432 ,
    pa180_p: -0.000290080640591335,
    pa141_p: -0.000371731800974898 ,
    pa161_p: -0.00564538392418283 ,
    pa181_p: -0.00216477129604764 ,
    pe120_p: -0.00234345993674653 ,
    pe140_p: -0.0025598346977071 ,
    pe160_p: -0.055748823259178,
    pe180_p: -0.00137834776769998,
    pe141_p: -0.00821977537556325,
    pe161_p: -0.0255682154771119,
    pe181_p: -0.0102822273478348 ,
    pg120_p: -0.000703620745734723,
    pg140_p: -3.03207570481577E-05,
    pg160_p: -0.0122303684865845,
    pg180_p: -0.000418944804169265,
    pg141_p: -0.00290595573235281,
    pg161_p: -0.00862596402374158,
    pg181_p: -0.00312454317572049,
})

print(r)

PLIPIDS: 0.000503555042155509 pa120_p + 0.000371731800974898 pa141_p + 0.00207597350252432 pa160_p + 0.00564538392418283 pa161_p + 0.000290080640591335 pa180_p + 0.00216477129604764 pa181_p + 0.00234345993674653 pe120_p + 0.0025598346977071 pe140_p + 0.00821977537556325 pe141_p + 0.055748823259178 pe160_p + 0.0255682154771119 pe161_p + 0.00137834776769998 pe180_p + 0.0102822273478348 pe181_p + 0.000703620745734723 pg120_p + 3.03207570481577e-05 pg140_p + 0.00290595573235281 pg141_p + 0.0122303684865845 pg160_p + 0.00862596402374158 pg161_p + 0.000418944804169265 pg180_p + 0.00312454317572049 pg181_p <=> 0.1483 lipid_NM_p


In [147]:
model.slim_optimize()

2.2345042552092766

In [148]:
r=model.reactions.get_by_id('PLIPIDS_ESS')
print(r)

PLIPIDS_ESS: 0.001857 pa120_p + 0.001157 pa140_p + 0.001202 pa141_p + 0.003838 pa160_p + 0.006675 pa161_p + 0.000484 pa180_p + 0.0008971 pa181_p + 0.001551 pg120_p + 0.001283 pg140_p + 0.001016 pg141_p + 0.001668 pg141b_p + 0.003285 pg160_p + 0.002712 pg160a_p + 0.001644 pg160b_p + 0.006129 pg160c_p + 0.006099 pg161_p + 0.000418 pg180_p + 0.000774 pg181_p <=> 0.04269 esslipid_NM_p


In [149]:
r= model.reactions.get_by_id('PLIPIDS_ESS')

pg141b_p= model.metabolites.get_by_id('pg141b_p')
pg160a_p= model.metabolites.get_by_id('pg160a_p')
pg160b_p= model.metabolites.get_by_id('pg160b_p')
pg160c_p= model.metabolites.get_by_id('pg160c_p')

pe141b_p= model.metabolites.get_by_id('pe141b_p')
pe160a_p= model.metabolites.get_by_id('pe160a_p')
pe160c_p= model.metabolites.get_by_id('pe160c_p')
pe160e_p= model.metabolites.get_by_id('pe160e_p')
pg141b_p= model.metabolites.get_by_id('pg141b_p')
pg160a_p= model.metabolites.get_by_id('pg160a_p')
pg160b_p= model.metabolites.get_by_id('pg160b_p')
pg160c_p= model.metabolites.get_by_id('pg160c_p')

pa120_p= model.metabolites.get_by_id('pa120_p')
pa140_p= model.metabolites.get_by_id('pa140_p')
pa141_p= model.metabolites.get_by_id('pa141_p')
pa160_p= model.metabolites.get_by_id('pa160_p')
pa161_p= model.metabolites.get_by_id('pa161_p')
pa180_p= model.metabolites.get_by_id('pa180_p')
pa181_p= model.metabolites.get_by_id('pa181_p')

pe120_p= model.metabolites.get_by_id('pe120_p')
pe140_p= model.metabolites.get_by_id('pe140_p')
pe141_p= model.metabolites.get_by_id('pe141_p')
pe160_p= model.metabolites.get_by_id('pe160_p')
pe161_p= model.metabolites.get_by_id('pe161_p')
pe180_p= model.metabolites.get_by_id('pe180_p')
pe181_p= model.metabolites.get_by_id('pe181_p')

pg120_p= model.metabolites.get_by_id('pg120_p')
pg140_p= model.metabolites.get_by_id('pg140_p')
pg141_p= model.metabolites.get_by_id('pg141_p')
pg160_p= model.metabolites.get_by_id('pg160_p')
pg161_p= model.metabolites.get_by_id('pg161_p')
pg180_p= model.metabolites.get_by_id('pg180_p')
pg181_p= model.metabolites.get_by_id('pg181_p')


r.add_metabolites({
 
    pa120_p:  0.001857, 
    pa140_p: 0.001157 ,
    pa141_p: 0.001202 ,
    pa160_p: 0.003838,
    pa161_p: 0.006675,
    pa180_p:  0.000484,
    pa181_p: 0.0008971, 

    pg120_p: 0.001551 ,
    pg140_p: 0.001283 ,
    pg160_p: 0.003285 ,
    pg160a_p:  0.002712, 
    pg160b_p: 0.001644 ,
    pg160c_p:  0.006129, 
    pg161_p: 0.006099 ,
    pg181_p: 0.000774 ,
    pg180_p: 0.000418 ,
    pg141_p: 0.001016 ,
    pg141b_p: 0.001668, })

  
r.add_metabolites({ 
    pa120_p: -0.000503555042155509,
    
    pa141_p: -0.000371731800974898 ,
    pa160_p: -0.00207597350252432 ,
    pa161_p: -0.00564538392418283 ,
    pa180_p: -0.000290080640591335,
    pa181_p: -0.00216477129604764 ,
  
    pg120_p: -0.000703620745734723,
    pg140_p: -3.03207570481577E-05,
    pg141_p: -0.00290595573235281,
    pg160_p: -0.0122303684865845,
    pg161_p: -0.00862596402374158,
    pg180_p: -0.000418944804169265,
    pg181_p: -0.00312454317572049, })

print(r)

PLIPIDS_ESS: 0.000503555042155509 pa120_p + 0.000371731800974898 pa141_p + 0.00207597350252432 pa160_p + 0.00564538392418283 pa161_p + 0.000290080640591335 pa180_p + 0.00216477129604764 pa181_p + 0.000703620745734723 pg120_p + 3.03207570481577e-05 pg140_p + 0.00290595573235281 pg141_p + 0.0122303684865845 pg160_p + 0.00862596402374158 pg161_p + 0.000418944804169265 pg180_p + 0.00312454317572049 pg181_p <=> 0.04269 esslipid_NM_p


In [150]:
model.slim_optimize()

2.2335018198084726

In [151]:
model.remove_reactions('AGPAT141b')
model.remove_reactions('AGPAT160b')
model.remove_reactions('AGPAT160c')
model.remove_reactions('AGPAT160e')




C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [152]:
model.slim_optimize()

2.233501819808473

In [153]:
model.remove_reactions('DASYN160a')
model.remove_reactions('DASYN160b')
model.remove_reactions('DASYN160c')
model.remove_reactions('DASYN160e')
model.remove_reactions('DASYN141b')

In [154]:
model.slim_optimize()

2.2335018198084695

In [155]:
model.remove_reactions('PGPP141b')
model.remove_reactions('PGPP160a')
model.remove_reactions('PGPP160b')
model.remove_reactions('PGPP160c')
model.remove_reactions('PGPP160e')

In [156]:
model.slim_optimize()

2.233501819808467

In [157]:
model.remove_reactions('PGPP160app')
model.remove_reactions('PGPP160bpp')
model.remove_reactions('PGPP160cpp')
model.remove_reactions('PGPP160epp')
model.remove_reactions('PGPP141bpp')

In [158]:
model.slim_optimize()

2.2335018198084664

In [159]:
model.remove_reactions('PGSA160a')
model.remove_reactions('PGSA160b')
model.remove_reactions('PGSA160c')
model.remove_reactions('PGSA160e')
model.remove_reactions('PGSA141b')

In [160]:
model.slim_optimize()

2.2335018198084677

In [161]:
model.remove_reactions('PLIPA1A160a')
model.remove_reactions('PLIPA1A160b')
model.remove_reactions('PLIPA1A160c')
model.remove_reactions('PLIPA1A160e')
model.remove_reactions('PLIPA1A141b')

In [162]:
model.slim_optimize()

2.2335018198084744

In [163]:
model.remove_reactions('PLIPA1E160a')
model.remove_reactions('PLIPA1E160b')
model.remove_reactions('PLIPA1E160c')
model.remove_reactions('PLIPA1E160e')
model.remove_reactions('PLIPA1E141b')

In [164]:
model.slim_optimize()

2.233501819808475

In [165]:
model.remove_reactions('PLIPA1G160a')
model.remove_reactions('PLIPA1G160b')
model.remove_reactions('PLIPA1G160c')
model.remove_reactions('PLIPA1G160e')
model.remove_reactions('PLIPA1G141b')

In [166]:
model.slim_optimize()

2.2335018198084593

In [167]:
model.remove_reactions('PSD160a')
model.remove_reactions('PSD160b')
model.remove_reactions('PSD160c')
model.remove_reactions('PSD160e')
model.remove_reactions('PSD141b')

In [168]:
model.slim_optimize()

2.2335018198084695

In [169]:
model.remove_reactions('PSSA160a')
model.remove_reactions('PSSA160b')
model.remove_reactions('PSSA160c')
model.remove_reactions('PSSA160e')
model.remove_reactions('PSSA141b')

In [170]:
model.slim_optimize()

2.2335018198084753

In [171]:
model.remove_reactions('2AGPA181atipp')

In [172]:
model.slim_optimize()

2.233501819808475

In [173]:
model.remove_reactions('PA141babcpp')
model.remove_reactions('PA160aabcpp')
model.remove_reactions('PA160babcpp')
model.remove_reactions('PA160cabcpp')
model.remove_reactions('PA160eabcpp')
model.remove_reactions('PE160aabcpp')
model.remove_reactions('PE160babcpp')
model.remove_reactions('PE160cabcpp')
model.remove_reactions('PE160eabcpp')
model.remove_reactions('PG141babcpp')
model.remove_reactions('PG160aabcpp')
model.remove_reactions('PG160babcpp')
model.remove_reactions('PG160cabcpp')
model.remove_reactions('PG160eabcpp')

In [174]:
model.remove_reactions('PGP141babcpp')
model.remove_reactions('PGP160aabcpp')
model.remove_reactions('PGP160babcpp')
model.remove_reactions('PGP160cabcpp')
model.remove_reactions('PGP160eabcpp')

In [175]:
model.slim_optimize()

2.233501819808467

In [176]:
#GARFT2
r = model.reactions.get_by_id('GARFT2')
print(r.check_mass_balance())
r

{'charge': -2.0, 'H': -2.0}


Reaction identifier,GARFT2
Name,R_GARFT2
Memory address,0x020f45ddd7c0
Stoichiometry,gar_c + h2o_c + methf_c --> fgam_c + thf_c N1-5-Phospho-D-ribosyl-glycinamide + H2O + 5-10-Methenyltetrahydrofolate --> N2-Formyl-N1-5-phospho-D-ribosyl-glycinamide + 5-6-7-8-Tetrahydrofolate
GPR,NGO1224
Lower bound,0.0
Upper bound,1000.0


In [177]:
h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2.0,
})

print(r)
print(r.check_mass_balance())

GARFT2: gar_c + h2o_c + methf_c --> fgam_c + 2.0 h_c + thf_c
{}


### Curate LOS synthesis

In [178]:
#rename LPS_NM to LOS_gc

model.metabolites.get_by_id('lps_NM_c').id='los_GC_c'

In [179]:
model.metabolites.get_by_id('los_GC_c').name='los_GC_c'

In [180]:
#rename LPS_NM to LOS_GC
#Siaylation is not required for synthesis of LOS. Changed the "LOS" 
# designation to the the final unsialylated component. Renamed the sialylated LOS to "Sialylated LOS".
model.metabolites.get_by_id('los_GC_c').id='sialylated_los_GC_c'
model.metabolites.get_by_id('sialylated_los_GC_c').name='sialylated_los_GC_c'

model.metabolites.get_by_id('galgnagalicolipa_c').id='los_GC_c'
model.metabolites.get_by_id('los_GC_c').name='los_GC_c'

In [181]:
#Remove export of LOS and associated extracellular LOS components.
model.remove_reactions('LPS3tex')
model.remove_reactions('LPS8tex')

model.remove_metabolites(model.metabolites.get_by_id('galicolipa_e'))
model.remove_metabolites(model.metabolites.get_by_id('lps_NM_e'))

In [182]:
#For GLCTR1 - Formula for icolipa_c is incorrect. When glc is added to peagnahhlipa_c, the previous reaction did not add the oxygens. 
#The corresponding reactions in LOS synthesis all are also missing these 5 oxygens. This is corrected here.
#Used structure diagrams here PMID: 11496013 and here PMID: 30873172 to determine proper formulas. 

#The reaction GLCTR1 corresponds to a bigg ID for a slightly different reaction in E.coli LPS biosynthesis. 
#This was left in place, but may need to be changed at a later time to avoid confusion.

#Additionally for SIALT1 - LPS is sialylated, the NeuAC was not actually added to the structure for LPS.  
#This is corrected here. Note this sialylation is on the alpha chain of LOS only, which is more common. Not on the beta chain.

r1 = model.reactions.get_by_id('GLCTR1')
r2 = model.reactions.get_by_id('SIALT1')
print(r1)
print(r1.check_mass_balance())
print(r2)
print(r2.check_mass_balance())

GLCTR1: peagnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c
{'charge': -5.0, 'H': 2.0, 'C': 2.0, 'N': -1.0, 'O': 7.0, 'P': -1.0}
SIALT1: cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c
{'C': -11.0, 'H': -18.0, 'N': -1.0, 'O': -8.0}


In [183]:
#The more common isoform for Gc is 4HEXG. This has the addition of a glc on both the alpha and the beta chain.
#The NM model did not have this glc on the beta chain and instead has a PEA. The PEA and glc on the beta chain are mutually exclusive. 
# Added reaction GLCTR2 to add a glc to the beta chain. 

glcgnahhlipa_c= Metabolite(
    'glcgnahhlipa_c',
    formula='C132H232N3O61P2',
    name='M_glcgnahhlipa_c',
    compartment='cytosol')

udpg_c= model.metabolites.get_by_id('udpg_c')
udp_c = model.metabolites.get_by_id('udp_c')
h_c = model.metabolites.get_by_id('h_c')
gnahhlipa_c=model.metabolites.get_by_id('gnahhlipa_c')



GLCTR2 = cobra.Reaction('GLCTR2')
GLCTR2.notes = {}
GLCTR2.name = 'R_GLCTR2'
GLCTR2.gene_reaction_rule = 'NGO2072'
GLCTR2.lower_bound = 0.
GLCTR2.upper_bound = 1000.
GLCTR2.add_metabolites({
    gnahhlipa_c: -1.0,
    glcgnahhlipa_c: 1.0,
    udpg_c: -1.0,
    udp_c: 1.0,
    h_c: 1,
})

model.add_reactions([GLCTR2])

r= model.reactions.get_by_id('GLCTR2')
print(r)
print(r.check_mass_balance())

GLCTR2: gnahhlipa_c + udpg_c --> glcgnahhlipa_c + h_c + udp_c
{'charge': 5.0}


In [184]:
#Remove reaction for addition of phosphoethanolamine to beta chain
model.remove_reactions('PEAT1')

In [185]:
#Add Gal to end of Beta chain to complete that segment of 4HEXG by adding reaction 
galglcgnahhlipa_c= Metabolite(
    'galglcgnahhlipa_c',
    formula='C138H242N3O66P2',
    name='M_galglcgnahhlipa_c',
    compartment='cytosol')

udpgal_c= model.metabolites.get_by_id('udpgal_c')
udp_c = model.metabolites.get_by_id('udp_c')
h_c = model.metabolites.get_by_id('h_c')
glcgnahhlipa_c=model.metabolites.get_by_id('glcgnahhlipa_c')



GALT3 = cobra.Reaction('GALT3')
GALT3.notes = {}
GALT3.name = 'R_GALT3'
GALT3.gene_reaction_rule = 'NGO2072'
GALT3.lower_bound = 0.
GALT3.upper_bound = 1000.
GALT3.add_metabolites({
    glcgnahhlipa_c: -1.0,
    galglcgnahhlipa_c: 1.0,
    udpg_c: -1.0,
    udp_c: 1.0,
    h_c: 1,
})

model.add_reactions([GALT3])



In [186]:
r=model.reactions.get_by_id('GALT3')
print(r)
print(r.check_mass_balance())

GALT3: glcgnahhlipa_c + udpg_c --> galglcgnahhlipa_c + h_c + udp_c
{}


In [187]:
#Create icolipa from the newly created 4HEXG Beta chain.
r = model.reactions.get_by_id('GLCTR1')

peagnahhlipa_c=model.metabolites.get_by_id('peagnahhlipa_c')

r.add_metabolites({
    peagnahhlipa_c: 1.0,
    galglcgnahhlipa_c: -1.0,
})


In [188]:
#Correct the icolipa formula and formulas for the alphachain components to reflect the new 4HEXG beta chain
model.metabolites.get_by_id('icolipa_c').formula= 'C144H252N3O71P2'   
model.metabolites.get_by_id('galicolipa_c').formula= 'C150H262N3O76P2'
model.metabolites.get_by_id('gnagalicolipa_c').formula= 'C158H276N4O81P2'
model.metabolites.get_by_id('los_GC_c').formula= 'C164H286N4O86P2'

model.metabolites.get_by_id('sialylated_los_GC_c').formula= 'C175H304N5O94P2'

print(r)
print(r.check_mass_balance())


GLCTR1: galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c
{'charge': -5.0}


In [189]:
#Add metabolite charges for los synthesis
model.metabolites.galglcgnahhlipa_c.charge=-5
model.metabolites.peagnahhlipa_c.charge=-10
model.metabolites.sialylated_los_GC_c.charge=-4
model.metabolites.los_GC_c.charge=-4
model.metabolites.galicolipa_c.charge=-5
model.metabolites.gnagalicolipa_c.charge=-4
model.metabolites.icolipa_c.charge=-5
model.metabolites.glcgnahhlipa_c.charge=-5
model.metabolites.gnahhlipa_c.charge=-5
model.metabolites.glcgnahhlipa_c.charge=-5

model.metabolites.gnahhlipa_c.charge=-5

In [190]:
#Check all the steps of LOS synthesis to ensure its balanced properly. 
#SIALT1
r = model.reactions.get_by_id('SIALT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,SIALT1
Name,R_SIALT1
Memory address,0x020f46053fd0
Stoichiometry,cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c CMP-N-acetyl neuraminate + los_GC_c --> CMP + sialylated_los_GC_c
GPR,NGO1081
Lower bound,0.0
Upper bound,1000.0


In [191]:
#GALT2
r = model.reactions.get_by_id('GALT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT2
Name,R_GALT2
Memory address,0x020f45dd0bb0
Stoichiometry,gnagalicolipa_c + udpgal_c --> h_c + los_GC_c + udp_c glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A + UDPgalactose --> H + los_GC_c + UDP
GPR,NGO2156
Lower bound,0.0
Upper bound,1000.0


In [192]:
#GNAT2
r = model.reactions.get_by_id('GNAT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GNAT2
Name,R_GNAT2
Memory address,0x020f45e0e310
Stoichiometry,"galicolipa_c + uacgam_c --> gnagalicolipa_c + udp_c galactosyl-inner-core-oligosaccharide-lipid-A, also mature lps, immunotype 8 + UDP-N-acetyl-D-glucosamine --> glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A + UDP"
GPR,NGO2158
Lower bound,0.0
Upper bound,1000.0


In [193]:
#GALT1
r = model.reactions.get_by_id('GALT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT1
Name,R_GALT1
Memory address,0x020f45dd0af0
Stoichiometry,"icolipa_c + udpgal_c --> galicolipa_c + h_c + udp_c inner-core-oligosaccharide-lipid-A + UDPgalactose --> galactosyl-inner-core-oligosaccharide-lipid-A, also mature lps, immunotype 8 + H + UDP"
GPR,NGO2159
Lower bound,0.0
Upper bound,1000.0


In [194]:
#GALT3
r = model.reactions.get_by_id('GALT3')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT3
Name,R_GALT3
Memory address,0x020f45b2a670
Stoichiometry,glcgnahhlipa_c + udpg_c --> galglcgnahhlipa_c + h_c + udp_c M_glcgnahhlipa_c + UDPglucose --> M_galglcgnahhlipa_c + H + UDP
GPR,NGO2072
Lower bound,0.0
Upper bound,1000.0


In [195]:
#GLCTR2
r = model.reactions.get_by_id('GLCTR2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GLCTR2
Name,R_GLCTR2
Memory address,0x020f45b2a730
Stoichiometry,gnahhlipa_c + udpg_c --> glcgnahhlipa_c + h_c + udp_c glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDPglucose --> M_glcgnahhlipa_c + H + UDP
GPR,NGO2072
Lower bound,0.0
Upper bound,1000.0


In [196]:
#GLCTR1
r = model.reactions.get_by_id('GLCTR1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GLCTR1
Name,R_GLCTR1
Memory address,0x020f45dea850
Stoichiometry,galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c M_galglcgnahhlipa_c + UDPglucose --> H + inner-core-oligosaccharide-lipid-A + UDP
GPR,NGO1353
Lower bound,0.0
Upper bound,1000.0


In [197]:
#GNAT1
r = model.reactions.get_by_id('GNAT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GNAT1
Name,R_GNAT1
Memory address,0x020f45e0ef70
Stoichiometry,hhlipa_c + uacgam_c --> gnahhlipa_c + udp_c heptosyl-heptosyl-kdo2-lipidA + UDP-N-acetyl-D-glucosamine --> glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDP
GPR,NGO1354
Lower bound,0.0
Upper bound,1000.0


In [198]:
#HEPT2
r = model.reactions.get_by_id('HEPT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,HEPT2
Name,R_HEPT2
Memory address,0x020f45e34610
Stoichiometry,adphep_LD_c + hlipa_c --> adp_c + h_c + hhlipa_c ADP-L-glycero-D-manno-heptose + heptosyl-kdo2-lipidA --> ADP + H + heptosyl-heptosyl-kdo2-lipidA
GPR,NGO0987
Lower bound,0.0
Upper bound,1000.0


In [199]:
#HEPT1
r = model.reactions.get_by_id('HEPT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,HEPT1
Name,R_HEPT1
Memory address,0x020f45e34550
Stoichiometry,adphep_LD_c + lipa_c --> adp_c + h_c + hlipa_c ADP-L-glycero-D-manno-heptose + KDO-2-lipid-A --> ADP + H + heptosyl-kdo2-lipidA
GPR,NGO1934
Lower bound,0.0
Upper bound,1000.0


In [200]:
#All of LOS synthesis looks correct. Continue checking/ correcting other reactions. 

### Additional Curation of mass/charge balance

In [201]:
model.slim_optimize()

2.2335018198084664

In [202]:
#HMT
r = model.reactions.get_by_id('HMT')
print(r.check_mass_balance())
r

{'charge': -1.0, 'H': -1.0}


Reaction identifier,HMT
Name,R_HMT
Memory address,0x020f45e41880
Stoichiometry,amet_c + his_L_c --> ahcys_c + mhis_c S-Adenosyl-L-methionine + L-Histidine --> S-Adenosyl-L-homocysteine + N(pi)-Methyl-L-histidine
GPR,NGO0746
Lower bound,0.0
Upper bound,1000.0


In [203]:
#No evidence that this reaction exists in Gc. 
#It forms a dead end metabolite that is not used elsewhere in the network.
#The gene (ngo0746) does possibly encode a methyltransferase, but this specific reaction doesn't appear to be found in any other models of bacteria, and isn't included in the CARVEME model for Gc. 
#As such, I am deleting the reaction. 

model.remove_reactions('HMT')

In [204]:
#MAc3
r = model.reactions.get_by_id('MAc3')
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x020f45e80370
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534 and Orphan
Lower bound,0.0
Upper bound,1000.0


In [205]:
# Both Gc and Nm acylate their peptidoglycan. PMCID: PMC1231103
print(model.metabolites.get_by_id('murein5p4p_p').formula)
print(model.metabolites.get_by_id('murein5p4pOAc_p').formula)


C77H119N15O41
C79H123N15O43


In [206]:
#In the formula for murein5p4pOAc, the oxygen that gets added during acylation was forgotten. This is added here. 
model.metabolites.get_by_id('murein5p4pOAc_p').formula= 'C79H123N15O43'  


In [207]:
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x020f45e80370
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534 and Orphan
Lower bound,0.0
Upper bound,1000.0


In [208]:
#peptidoglycan acylation has been shown to occur due to NGO0533 and NGO0534 (NMB1274 and NMB1273) aka PatA and PatB aka PacA and PacB. 
#THe third protein, proposed as an orphan, is apeI which actually is an esterase. 
#(PMID: 23373517)
#I correct the GPR rules for the MAc1 - MAc8 reactions here. 

NGO0533 =model.genes.get_by_id('NGO0533')
NGO0534= model.genes.get_by_id('NGO0534')
model.reactions.get_by_id('MAc1').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc2').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc3').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc4').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc5').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc6').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc7').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc8').gene_reaction_rule ='NGO0533 and NGO0534'
r

Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x020f45e80370
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [209]:
#Most of the PG in the cell wall is tetrapeptide in Gc PMCID: PMC551031 and is O-acytelated. 
#However tripeptide monomer gets released extracellularly and can cause immune responses through NOD1.
#Cleavage of the tetrapeptide to tripeptide, is driven by ldcA (NGO1274) (PMCID: PMC5646250)

#Confirmed, that tripeptide monomer is produced in the model (anhgm3p_p).



# Gc and neisseria in general use tetrapeptide PG in the cell wall, rather than tripeptide which is used in E.coli. 
# This model uses the "most common" o-acetylated tetrapeptide trimer to make PG for biomass. (murein4px4px4pOAc and murein 4p4p4pOAc).
#The model uses murein5p5p5p to make murein 4p4p4p (MDDCP6pp) which is then acetylated (MAc1).
#Its not clear how this would happen in nature since these reactions typically work on crosslinked peptidoglycan, not on uncrosslinked. 


r = model.reactions.get_by_id('PEPGLY')
print(r.check_mass_balance())
r

murein4p4p4pOAc_p=model.metabolites.get_by_id('murein4p4p4pOAc_p')
murein4px4px4p_p = model.metabolites.get_by_id('murein4px4px4p_p')

r.add_metabolites({
    murein4p4p4pOAc_p: 0.008334,
    murein4px4px4p_p : -0.008334,
})

print(r.check_mass_balance())
r






{'charge': 0.033336000000000005, 'C': -1.5695700000000001, 'H': -2.433528, 'N': -0.24168599999999998, 'O': -0.8500679999999999}
{'charge': 0.08334, 'C': -1.552902, 'H': -2.341854, 'N': -0.29169, 'O': -0.816732}


Reaction identifier,PEPGLY
Name,R_PEPGLY
Memory address,0x020f45f5fdc0
Stoichiometry,0.005556 murein4px4px4pOAc_p + 0.008334 murein4px4px4p_p <=> 0.1389 pepglycan_NM_p 0.005556 three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain-O_acytelated + 0.008334 three-...
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [210]:
#MAc7 and MAc8 which produces murein4p4p4pdOac and murein 4p4p4ptOAc respectively, where altered to use murein 4px4px4p as the base instead.
r = model.reactions.get_by_id('MAc7')
print(r.check_mass_balance())
r


murein4px4px4pdOAc_p= Metabolite(
    'murein4px4px4pdOAc_p',
    formula='C115H175N21O62',
    name='M_murein4px4px4pdOAc_p',
    compartment='periplasm')

murein4px4px4ptOAc_p= Metabolite(
    'murein4px4px4ptOAc_p',
    formula='C117H179N21O64',
    name='M_murein4px4px4ptOAc_p',
    compartment='periplasm')

murein4p4p4pOAC_p=model.metabolites.get_by_id('murein4p4p4pOAc_p')
murein4px4px4pOAc_p = model.metabolites.get_by_id('murein4px4px4pOAc_p')
murein4p4p4pdOAc_p=model.metabolites.get_by_id('murein4p4p4pdOAc_p')

murein4p4p4ptOAc_p=model.metabolites.get_by_id('murein4p4p4ptOAc_p')


r.add_metabolites({
    murein4p4p4pOAc_p: 1,
    murein4px4px4pOAc_p : -1,
    murein4p4p4pdOAc_p: -1,
    murein4px4px4pdOAc_p : 1,
})

print(r.check_mass_balance())
r



{}
{'charge': 6.0}


Reaction identifier,MAc7
Name,R_MAc7
Memory address,0x020f45e8c370
Stoichiometry,accoa_c + h2o_p + murein4px4px4pOAc_p --> coa_c + murein4px4px4pdOAc_p Acetyl-CoA + H2O + three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain-O_acytelated -->...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [211]:
r = model.reactions.get_by_id('MAc8')
print(r.check_mass_balance())
r

r.add_metabolites({

    murein4p4p4pdOAc_p: 1,
    murein4px4px4pdOAc_p : -1,
    murein4p4p4ptOAc_p: -1,
    murein4px4px4ptOAc_p : 1,
})




{}


In [212]:
#And I am deleting MLTGY5pp as the lytic transglycosylase should also only work on crosslinked peptidoglycan, since its catalytic activity is to remove those crosslinks.
model.remove_reactions('MLTGY5pp')

In [213]:
#Rename pepglycan metabolite
model.metabolites.pepglycan_NM_p.id="pepglycan_p"
model.metabolites.pepglycan_p.name='M_pepglycan_p'


In [214]:
model.optimize()

,fluxes,reduced_costs
12DGR120tipp,0.000000,0.0
12DGR140tipp,0.000000,0.0
12DGR141tipp,0.000000,0.0
12DGR160tipp,0.000000,0.0
12DGR161tipp,0.000000,0.0
...,...,...
EX_cystine_L_,0.000000,0.0
CYSR,0.000000,-0.0
EX_apoACP_c_,-0.082089,0.0
GLCTR2,0.000000,0.0


In [215]:
#The third protein encoded in the operon is ApeI which de-O-acetylates and has broad acting esterase activity. 
#(PMID: 23373517)
#The original model removed the esterase activity to prevent futile cycling. 
#The reactions are kept the same here. 


In [216]:
model.slim_optimize()

2.2337742792060107

In [217]:
r = model.reactions.get_by_id('NDPK9')
print(r.check_mass_balance())
r

{}


Reaction identifier,NDPK9
Name,R_NDPK9
Memory address,0x020f45ee3fd0
Stoichiometry,atp_c + didp_c <=> adp_c + ditp_c ATP + dIDP <=> ADP + dITP
GPR,NGO0597
Lower bound,-1000.0
Upper bound,1000.0


In [218]:
r.name = 'NADPK10'
r.id ='NADPK10'

model.metabolites.get_by_id('didp_c').formula= 'C10H11N4O10P2'

print(r.check_mass_balance())
r


{}


Reaction identifier,NADPK10
Name,NADPK10
Memory address,0x020f45ee3fd0
Stoichiometry,atp_c + didp_c <=> adp_c + ditp_c ATP + dIDP <=> ADP + dITP
GPR,NGO0597
Lower bound,-1000.0
Upper bound,1000.0


In [219]:
#No evidence for NGO1881 performing this reaction or for the mening homolog performing the reaction. 
model.reactions.NDPK1.gene_reaction_rule= 'NGO0597'
model.reactions.NDPK5.gene_reaction_rule= 'NGO0597'
model.reactions.NDPK8.gene_reaction_rule= 'NGO0597'

In [220]:
#NTD13
r = model.reactions.get_by_id('NTD13')
print(r.check_mass_balance())
r


{}


Reaction identifier,NTD13
Name,R_NTD13
Memory address,0x020f45f07e50
Stoichiometry,h2o_c + nicrnt_c --> nicrns_c + pi_c H2O + Nicotinate-D-ribonucleotide --> Nicotinate D-ribonucleoside + Phosphate
GPR,NGO1058
Lower bound,0.0
Upper bound,1000.0


In [221]:
#changed nicrns_c formula based on Bigg metabolite formula. 
model.metabolites.get_by_id('nicrns_c').formula= 'C11H13NO6'

In [222]:
print(r.check_mass_balance())
r

{}


Reaction identifier,NTD13
Name,R_NTD13
Memory address,0x020f45f07e50
Stoichiometry,h2o_c + nicrnt_c --> nicrns_c + pi_c H2O + Nicotinate-D-ribonucleotide --> Nicotinate D-ribonucleoside + Phosphate
GPR,NGO1058
Lower bound,0.0
Upper bound,1000.0


In [223]:
#PHEMEFErel
r = model.reactions.get_by_id('PHEMEFErel')
print(r.check_mass_balance())
r

{'charge': 4.0, 'H': 3.0}


Reaction identifier,PHEMEFErel
Name,R_PHEMEFErel
Memory address,0x020f45fa8370
Stoichiometry,3.0 h_c + 3.0 nadph_c + 3.0 o2_c + pheme_c --> biliverd_c + co_c + fe2_c + 3.0 h2o_c + 3.0 nadp_c 3.0 H + 3.0 Nicotinamide-adenine-dinucleotide-phosphate--reduced + 3.0 O2 + Protoheme --> Biliverdin + Carbon monoxide + Fe2 + 3.0 H2O + 3.0 Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO1318
Lower bound,0.0
Upper bound,1000.0


In [224]:
#Product of a simplified, 3 step reaction that requires 2 NADPHs. PMCID: PMC111422. 
#This reaction is described in kegg as requiring 3 "electron donors" and 3"electron acceptors, which is likely the cause of the error. https://www.genome.jp/entry/R11816


model.metabolites.get_by_id('biliverd_c').formula= 'C33H33N4O6'
NADPH_c =model.metabolites.get_by_id('nadph_c')
nadp_c =model.metabolites.get_by_id('nadp_c')

r.add_metabolites({
    NADPH_c: -3,
    nadp_c: 3.0
})

print(r.check_mass_balance())
r

{'charge': 7.0}


Reaction identifier,PHEMEFErel
Name,R_PHEMEFErel
Memory address,0x020f45fa8370
Stoichiometry,3.0 h_c + 6.0 nadph_c + 3.0 o2_c + pheme_c --> biliverd_c + co_c + fe2_c + 3.0 h2o_c + 6.0 nadp_c 3.0 H + 6.0 Nicotinamide-adenine-dinucleotide-phosphate--reduced + 3.0 O2 + Protoheme --> Biliverdin + Carbon monoxide + Fe2 + 3.0 H2O + 6.0 Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO1318
Lower bound,0.0
Upper bound,1000.0


In [225]:
#PMCOAS
r = model.reactions.get_by_id('PMCOAS')
print(r.check_mass_balance())
r
#Okay to leave unbalanced. THis reaction is a simplification of a multistep ACP dependent reaction that isn't well understood. 

{'charge': -1.0, 'C': 7.0, 'H': 9.0, 'O': 3.0}


Reaction identifier,PMCOAS
Name,R_PMCOAS
Memory address,0x020f45fd9f10
Stoichiometry,coa_c --> pmcoa_c Coenzyme-A --> Pimeloyl-CoA
GPR,NGO1481 and NGO1725
Lower bound,0.0
Upper bound,1000.0


In [226]:
#SAMC
r = model.reactions.get_by_id('SAMC')
print(r.check_mass_balance())
r



{'charge': 1.0, 'H': 1.0}


Reaction identifier,SAMC
Name,R_SAMC
Memory address,0x020f46047940
Stoichiometry,amet_c --> ametam_c + co2_c S-Adenosyl-L-methionine --> S-Adenosylmethioninamine + CO2
GPR,Orphan
Lower bound,0.0
Upper bound,1000.0


In [227]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: -1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SAMC
Name,R_SAMC
Memory address,0x020f46047940
Stoichiometry,amet_c + h_c --> ametam_c + co2_c S-Adenosyl-L-methionine + H --> S-Adenosylmethioninamine + CO2
GPR,Orphan
Lower bound,0.0
Upper bound,1000.0


In [228]:
#SHSL2
r = model.reactions.get_by_id('SHSL2')
print(r.check_mass_balance())
r


{'charge': -1.0, 'H': -1.0}


Reaction identifier,SHSL2
Name,R_SHSL2
Memory address,0x020f46053bb0
Stoichiometry,h2s_c + suchms_c --> hcys_L_c + succ_c Hydrogen-sulfide + O-Succinyl-L-homoserine --> L-Homocysteine + Succinate
GPR,NGO1149
Lower bound,0.0
Upper bound,1000.0


In [229]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL2
Name,R_SHSL2
Memory address,0x020f46053bb0
Stoichiometry,h2s_c + suchms_c --> h_c + hcys_L_c + succ_c Hydrogen-sulfide + O-Succinyl-L-homoserine --> H + L-Homocysteine + Succinate
GPR,NGO1149
Lower bound,0.0
Upper bound,1000.0


In [230]:
#SHLS3
r = model.reactions.get_by_id('SHSL3')
print(r.check_mass_balance())
r


{'charge': -1.0, 'H': -1.0}


Reaction identifier,SHSL3
Name,R_SHSL3
Memory address,0x020f46053d00
Stoichiometry,achms_c + cys_L_c --> ac_c + cyst_L_c O-acetyl-L-homoserine + L-Cysteine --> Acetate + L-Cystathionine
GPR,NGO0386
Lower bound,0.0
Upper bound,1000.0


In [231]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL3
Name,R_SHSL3
Memory address,0x020f46053d00
Stoichiometry,achms_c + cys_L_c --> ac_c + cyst_L_c + h_c O-acetyl-L-homoserine + L-Cysteine --> Acetate + L-Cystathionine + H
GPR,NGO0386
Lower bound,0.0
Upper bound,1000.0


In [232]:
#SHLS5
r = model.reactions.get_by_id('SHSL5')
print(r.check_mass_balance())
r


{'charge': -1.0, 'H': -1.0}


Reaction identifier,SHSL5
Name,R_SHSL5
Memory address,0x020f46053f40
Stoichiometry,h2o_c + suchms_c --> 2obut_c + nh4_c + succ_c H2O + O-Succinyl-L-homoserine --> 2-Oxobutanoate + Ammonium + Succinate
GPR,NGO0386
Lower bound,0.0
Upper bound,1000.0


In [233]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL5
Name,R_SHSL5
Memory address,0x020f46053f40
Stoichiometry,h2o_c + suchms_c --> 2obut_c + h_c + nh4_c + succ_c H2O + O-Succinyl-L-homoserine --> 2-Oxobutanoate + H + Ammonium + Succinate
GPR,NGO0386
Lower bound,0.0
Upper bound,1000.0


In [234]:
#SHLS4
r = model.reactions.get_by_id('SHSL4')
print(r.check_mass_balance())
r


{'charge': -2.0, 'H': -2.0}


Reaction identifier,SHSL4
Name,R_SHSL4
Memory address,0x020f46053e20
Stoichiometry,achms_c + h_c + trdrd_c + tsul_c --> ac_c + hcys_L_c + so3_c + trdox_c O-acetyl-L-homoserine + H + Reduced-thioredoxin + Thiosulfate --> Acetate + L-Homocysteine + Sulfite + Oxidized-thioredoxin
GPR,NGO0386
Lower bound,0.0
Upper bound,1000.0


In [235]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL4
Name,R_SHSL4
Memory address,0x020f46053e20
Stoichiometry,achms_c + trdrd_c + tsul_c --> ac_c + h_c + hcys_L_c + so3_c + trdox_c O-acetyl-L-homoserine + Reduced-thioredoxin + Thiosulfate --> Acetate + H + L-Homocysteine + Sulfite + Oxidized-thioredoxin
GPR,NGO0386
Lower bound,0.0
Upper bound,1000.0


In [236]:
#TFFE
r = model.reactions.get_by_id('TFFE')
print(r.check_mass_balance())
r


{'charge': 3.0, 'Fe': 1.0}


Reaction identifier,TFFE
Name,R_TFFE
Memory address,0x020f460796a0
Stoichiometry,TfFe3_e --> Tf_e + 2.0 fe3_p Transferrin --> Transferrin residue + 2.0 Fe3
GPR,NGO1496 and NGO1495 and NGO1379 and NGO1378 and (Blank or NGO1377) and NGO0217 and NGO0216 and...
Lower bound,0.0
Upper bound,1000.0


In [237]:
#Although transferrin holds two irons, Gc is only capable of stripping one off at a time. So transferrin is made to produce a single iron here. 
fe3_p =model.metabolites.get_by_id('fe3_p')

r.add_metabolites({
    fe3_p: -1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,TFFE
Name,R_TFFE
Memory address,0x020f460796a0
Stoichiometry,TfFe3_e --> Tf_e + fe3_p Transferrin --> Transferrin residue + Fe3
GPR,NGO1496 and NGO1495 and NGO1379 and NGO1378 and (Blank or NGO1377) and NGO0217 and NGO0216 and...
Lower bound,0.0
Upper bound,1000.0


In [238]:
#TRDR1
#synthesis of thioredoxin. Protein based component.
r = model.reactions.get_by_id('TRDR1')
print(r.check_mass_balance())
r


{'X': 1.0}


Reaction identifier,TRDR1
Name,R_TRDR1
Memory address,0x020f46093b50
Stoichiometry,h_c + nadph_c --> nadp_c + trdrd_c H + Nicotinamide-adenine-dinucleotide-phosphate--reduced --> Nicotinamide-adenine-dinucleotide-phosphate + Reduced-thioredoxin
GPR,NGO0580 and NGO2124
Lower bound,0.0
Upper bound,1000.0


In [239]:
trdox_c =model.metabolites.get_by_id('trdox_c')

r.add_metabolites({
    trdox_c: -1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,TRDR1
Name,R_TRDR1
Memory address,0x020f46093b50
Stoichiometry,h_c + nadph_c + trdox_c --> nadp_c + trdrd_c H + Nicotinamide-adenine-dinucleotide-phosphate--reduced + Oxidized-thioredoxin --> Nicotinamide-adenine-dinucleotide-phosphate + Reduced-thioredoxin
GPR,NGO0580 and NGO2124
Lower bound,0.0
Upper bound,1000.0


In [240]:
#Add thioredoxin import reaction. This is required to show "synthesis" of a protein based component. 
#Integration of Biomass Formulations of Genome-Scale Metabolic Models with Experimental Data Reveals Universally Essential Cofactors in Prokaryotes
#Joana C Xavier, Kiran Raosaheb Patil, Isabel Rocha
#PMID: 27939572 PMCID: PMC5249239

trdrd_c = model.metabolites.get_by_id('trdrd_c')

EX_trdrd_c_ = cobra.Reaction('EX_trdrd_c_')
EX_trdrd_c_.gene_reaction_rule = ''
EX_trdrd_c_.lower_bound = 0.
EX_trdrd_c_.upper_bound = 1000.
EX_trdrd_c_.add_metabolites({
    trdrd_c: -1.0,
})

model.add_reactions([EX_trdrd_c_])


In [241]:
#TSULST
#synthesis of thioredoxin. Protein based component. 
r = model.reactions.get_by_id('TSULST')
print(r.check_mass_balance())
r


{'charge': -2.0, 'H': -2.0}


Reaction identifier,TSULST
Name,R_TSULST
Memory address,0x020f4609fa00
Stoichiometry,acser_c + h_c + trdrd_c + tsul_c <=> ac_c + cys_L_c + so3_c + trdox_c O-Acetyl-L-serine + H + Reduced-thioredoxin + Thiosulfate <=> Acetate + L-Cysteine + Sulfite + Oxidized-thioredoxin
GPR,NGO0340
Lower bound,-1000.0
Upper bound,1000.0


In [242]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,TSULST
Name,R_TSULST
Memory address,0x020f4609fa00
Stoichiometry,acser_c + trdrd_c + tsul_c <=> ac_c + cys_L_c + h_c + so3_c + trdox_c O-Acetyl-L-serine + Reduced-thioredoxin + Thiosulfate <=> Acetate + L-Cysteine + H + Sulfite + Oxidized-thioredoxin
GPR,NGO0340
Lower bound,-1000.0
Upper bound,1000.0


In [243]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

28 28 28


,Reactions,Charge,Masses
0,3OAS161a,1.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ADPT,-1.0,{'H': -1.0}
2,ANPRT,-1.0,{'H': -1.0}
3,ATPPRT,-1.0,{'H': -1.0}
4,BTS4,,{'S': 1.0}
5,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
6,GLUPRT,-1.0,{'H': -1.0}
7,GUAPRT,-1.0,{'H': -1.0}
8,DDPGA,2.0,{'H': 2.0}
9,HGFE,-8.0,{}


In [244]:
#DDPGA
r = model.reactions.get_by_id('DDPGA')
print(r.check_mass_balance())
r

{'charge': 2.0, 'H': 2.0}


Reaction identifier,DDPGA
Name,R_DDPGA
Memory address,0x020f45e288e0
Stoichiometry,4h2oglt_c --> glx_c + 2.0 h_c + pyr_c D-4-Hydroxy-2-oxoglutarate --> Glyoxylate + 2.0 H + Pyruvate
GPR,NGO0713
Lower bound,0.0
Upper bound,1000.0


In [245]:
#There are two extra Hs produced in this reaction due to changes in protonation for h2oglt/glx. 
#Bigg does not indicate hydrogens are produced in this reaction. Correcting to match bigg. 
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: -2,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,DDPGA
Name,R_DDPGA
Memory address,0x020f45e288e0
Stoichiometry,4h2oglt_c --> glx_c + pyr_c D-4-Hydroxy-2-oxoglutarate --> Glyoxylate + Pyruvate
GPR,NGO0713
Lower bound,0.0
Upper bound,1000.0


In [246]:
#P3H5CD
r = model.reactions.get_by_id('P3H5CD')
print(r.check_mass_balance())
r

{'charge': -1.0, 'H': -1.0}


Reaction identifier,P3H5CD
Name,R_P3H5CD
Memory address,0x020f45f2ddc0
Stoichiometry,1p3h5c_c + 2.0 h2o_c + nadp_c <=> e4hglu_c + h_c + nadph_c L-1-Pyrroline-3-hydroxy-5-carboxylate + 2.0 H2O + Nicotinamide-adenine-dinucleotide-phosphate <=> L-erythro-4-Hydroxyglutamate + H + Nicotinamide-adenine-dinucleotide-phosphate--reduced
GPR,NGO08225
Lower bound,-1000.0
Upper bound,1000.0


In [247]:
#The protonation of e4hglu was changed when correcting HGTA. This results in having an extra h consumed in P3H5CD.
#This is corrected here. 
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1.0,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,P3H5CD
Name,R_P3H5CD
Memory address,0x020f45f2ddc0
Stoichiometry,1p3h5c_c + 2.0 h2o_c + nadp_c <=> e4hglu_c + 2.0 h_c + nadph_c L-1-Pyrroline-3-hydroxy-5-carboxylate + 2.0 H2O + Nicotinamide-adenine-dinucleotide-phosphate <=> L-erythro-4-Hydroxyglutamate + 2.0 H + Nicotinamide-adenine-dinucleotide-phosphate--reduced
GPR,NGO08225
Lower bound,-1000.0
Upper bound,1000.0


In [248]:
model.reactions.get_by_id('3HAD161x')

Reaction identifier,3HAD161x
Name,R_3HAD161x
Memory address,0x020f45bdae80
Stoichiometry,"3hcpalm5eACP_c --> h2o_c + t3c5palmeACP_c M_3hcpalm5eACP_c --> H2O + Abbreviated name t3c5palmeACP_c trans-3-cis-5-palmitoleoyl-acyl-carrier-protein t3c5palmeACP_c trans-3-cis-5-palmitoleoyl-acyl-carrier-protein Name: Full name, dtype: object"
GPR,NGO1804
Lower bound,0.0
Upper bound,1000.0


In [249]:
model.metabolites.get_by_id('t3c5palmeACP_c').formula= 'C27H47N2O8PRS'

In [250]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

26 26 26


,Reactions,Charge,Masses
0,3OAS161a,1.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ADPT,-1.0,{'H': -1.0}
2,ANPRT,-1.0,{'H': -1.0}
3,ATPPRT,-1.0,{'H': -1.0}
4,BTS4,,{'S': 1.0}
5,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
6,GLUPRT,-1.0,{'H': -1.0}
7,GUAPRT,-1.0,{'H': -1.0}
8,HGFE,-8.0,{}
9,HXPRT,-1.0,{'H': -1.0}


In [251]:
r=model.reactions.get_by_id('MAc2')
model.metabolites.murein4px4px4pOAc_p.charge=-6
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc2
Name,R_MAc2
Memory address,0x020f45e80fa0
Stoichiometry,accoa_c + h2o_p + murein4px4px4p_p --> coa_c + murein4px4px4pOAc_p Acetyl-CoA + H2O + three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain --> Coenzyme-A + three-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [252]:
r=model.reactions.get_by_id('MAc4')
model.metabolites.murein4p4pOAc_p.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc4
Name,R_MAc4
Memory address,0x020f45e808b0
Stoichiometry,accoa_c + h2o_p + murein4p4p_p --> coa_c + murein4p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-middle-of-chain-O-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [253]:
r=model.reactions.get_by_id('MAc5')
model.metabolites.murein4p4pdOAc_p.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc5
Name,R_MAc5
Memory address,0x020f45e80eb0
Stoichiometry,accoa_c + h2o_p + murein4p4pOAc_p --> coa_c + murein4p4pdOAc_p Acetyl-CoA + H2O + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-middle-of-chain-O-acytelated --> Coenzyme-A + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [254]:
r=model.reactions.get_by_id('MAc6')
model.metabolites.murein4p3pOAc_p.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc6
Name,R_MAc6
Memory address,0x020f45e80d30
Stoichiometry,accoa_c + h2o_p + murein4p3p_p --> coa_c + murein4p3pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-tetrapeptide-and-tripeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-tetrapeptide-and-tripeptide-murein-units-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [255]:
r=model.reactions.get_by_id('MAc7')
model.metabolites.murein4px4px4pdOAc_p.charge=-6
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc7
Name,R_MAc7
Memory address,0x020f45e8c370
Stoichiometry,accoa_c + h2o_p + murein4px4px4pOAc_p --> coa_c + murein4px4px4pdOAc_p Acetyl-CoA + H2O + three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain-O_acytelated -->...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [256]:
r=model.reactions.get_by_id('MAc8')
model.metabolites.murein4px4px4ptOAc_p.charge=-6
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc8
Name,R_MAc8
Memory address,0x020f45e8c520
Stoichiometry,accoa_c + h2o_p + murein4px4px4pdOAc_p --> coa_c + murein4px4px4ptOAc_p Acetyl-CoA + H2O + M_murein4px4px4pdOAc_p --> Coenzyme-A + M_murein4px4px4ptOAc_p
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [257]:
r=model.reactions.get_by_id('3OAS161a')
r

#Added ACP and co2 to this reaction, as is seen in 3OAS161 for mass balance. 
r=model.reactions.get_by_id('3OAS161a')
ACP_c=model.metabolites.get_by_id('ACP_c')
co2_c=model.metabolites.get_by_id('co2_c')
r.add_metabolites({
    ACP_c:1,
    co2_c:1
      })
model.metabolites.get_by_id('3ocpalm5eACP_c').charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,3OAS161a
Name,R_3OAS161a
Memory address,0x020f45bfaf70
Stoichiometry,cmrs3eACP_c + h_c + malACP_c --> 3ocpalm5eACP_c + ACP_c + co2_c cis-Tetradec-3-enoyl-acyl-carrier-protein + H + Malonyl-acyl-carrier-protein --> 3-oxo-cis-palm-5-eoyl-acyl-carrier-protein + acyl-carrier-protein + CO2
GPR,NGO1763
Lower bound,0.0
Upper bound,1000.0


In [258]:
# Fix hydrogen balances

In [259]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

24 24 24


,Reactions,Charge,Masses
0,ADPT,-1.0,{'H': -1.0}
1,ANPRT,-1.0,{'H': -1.0}
2,ATPPRT,-1.0,{'H': -1.0}
3,BTS4,,{'S': 1.0}
4,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
5,GLUPRT,-1.0,{'H': -1.0}
6,GUAPRT,-1.0,{'H': -1.0}
7,HGFE,-8.0,{}
8,HXPRT,-1.0,{'H': -1.0}
9,MDDCP6pp,6.0,"{'H': 3.0, 'N': -6.0}"


In [260]:
#Bulk correct hydrogens
rxns=('METS', 'ORPT', 'PRPPS')

for rxn in rxns:
    r=model.reactions.get_by_id(rxn)
    r.add_metabolites({
        h_c: -1.0,
            })
    print(r.check_mass_balance())



#Bulk correct hydrogens
rxns=('ADPT', 'ANPRT','ATPPRT','GLUPRT','GUAPRT','HXPRT','MTHFR2','NAMNPP','NNDPR', 'PLIPA1E161a', 'PLIPA1G161a', 'UPPRT','XPPT','XPRT')

for rxn in rxns:
    r=model.reactions.get_by_id(rxn)
    r.add_metabolites({
       h_c: 1.0,
           })
    print(r.check_mass_balance())  

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [261]:
#what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge

7 7 7


,Reactions,Charge,Masses
0,BTS4,,{'S': 1.0}
1,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
2,HGFE,-8.0,{}
3,MDDCP6pp,6.0,"{'H': 3.0, 'N': -6.0}"
4,PHEMEFErel,7.0,{}
5,PMCOAS,-1.0,"{'C': 7.0, 'H': 9.0, 'O': 3.0}"
6,CYPOX,-2.0,{}


In [262]:
#All remaining imbalances are okay, due to either proteinaceous components, or respiratory functions

In [263]:
rpmi(model)
print ('RPMI',model.slim_optimize())
mdm(model)
print ('MDM', model.slim_optimize())
complete(model)
print('Complete media', model.slim_optimize())

RPMI 1.2426654074373995
MDM 1.156350324146482
Complete media 2.244317436496072


## 2c. New Biomass equation for Gc

In [264]:
r=model.reactions.get_by_id('Nm_Ess_biomass')
pa120_p=model.metabolites.get_by_id('pa120_p')
pa140_p=model.metabolites.get_by_id('pa140_p')
pa160_p=model.metabolites.get_by_id('pa160_p')
pa180_p=model.metabolites.get_by_id('pa180_p')
pa141_p=model.metabolites.get_by_id('pa141_p')
pa161_p=model.metabolites.get_by_id('pa161_p')
pa181_p=model.metabolites.get_by_id('pa181_p')
pe120_p=model.metabolites.get_by_id('pe120_p')
pe140_p=model.metabolites.get_by_id('pe140_p')
pe160_p=model.metabolites.get_by_id('pe160_p')
pe180_p=model.metabolites.get_by_id('pe180_p')
pe141_p=model.metabolites.get_by_id('pe141_p')
pe161_p=model.metabolites.get_by_id('pe161_p')
pe181_p=model.metabolites.get_by_id('pe181_p')
pg120_p=model.metabolites.get_by_id('pg120_p')
pg140_p=model.metabolites.get_by_id('pg140_p')
pg160_p=model.metabolites.get_by_id('pg160_p')
pg180_p=model.metabolites.get_by_id('pg180_p')
pg141_p=model.metabolites.get_by_id('pg141_p')
pg161a_p=model.metabolites.get_by_id('pg161_p')
pg181c_p=model.metabolites.get_by_id('pg181_p')
los_GC_c=model.metabolites.get_by_id('los_GC_c')
murein4px4px4pOAc_p=model.metabolites.get_by_id('murein4px4px4pOAc_p')
murein4p4p4pOAc_p=model.metabolites.get_by_id('murein4p4p4pOAc_p')
asp_L_c=model.metabolites.get_by_id('asp_L_c')
glu_L_c=model.metabolites.get_by_id('glu_L_c')
ser_L_c=model.metabolites.get_by_id('ser_L_c')
his_L_c=model.metabolites.get_by_id('his_L_c')
gly_c=model.metabolites.get_by_id('gly_c')
thr_L_c=model.metabolites.get_by_id('thr_L_c')
ala_L_c=model.metabolites.get_by_id('ala_L_c')
arg_L_c=model.metabolites.get_by_id('arg_L_c')
tyr_L_c=model.metabolites.get_by_id('tyr_L_c')
val_L_c=model.metabolites.get_by_id('val_L_c')
met_L_c=model.metabolites.get_by_id('met_L_c')
phe_L_c=model.metabolites.get_by_id('phe_L_c')
ile_L_c=model.metabolites.get_by_id('ile_L_c')
leu_L_c=model.metabolites.get_by_id('leu_L_c')
lys_L_c=model.metabolites.get_by_id('lys_L_c')
pro_L_c=model.metabolites.get_by_id('pro_L_c')
asn_L_c=model.metabolites.get_by_id('asn_L_c')
trp_L_c=model.metabolites.get_by_id('trp_L_c')
cys_L_c=model.metabolites.get_by_id('cys_L_c')
gln_L_c=model.metabolites.get_by_id('gln_L_c')
datp_c=model.metabolites.get_by_id('datp_c')
dttp_c=model.metabolites.get_by_id('dttp_c')
dgtp_c=model.metabolites.get_by_id('dgtp_c')
dctp_c=model.metabolites.get_by_id('dctp_c')
atp_c=model.metabolites.get_by_id('atp_c')
utp_c=model.metabolites.get_by_id('utp_c')
gtp_c=model.metabolites.get_by_id('gtp_c')
ctp_c=model.metabolites.get_by_id('ctp_c')
fthf_c=model.metabolites.get_by_id('10fthf_c')
ohph_c=model.metabolites.get_by_id('2ohph_c')
amet_c=model.metabolites.get_by_id('amet_c')
ca2_c=model.metabolites.get_by_id('ca2_c')
cl_c=model.metabolites.get_by_id('cl_c')
coa_c=model.metabolites.get_by_id('coa_c')
cobalt2_c=model.metabolites.get_by_id('cobalt2_c')
cu2_c=model.metabolites.get_by_id('cu2_c')
fad_c=model.metabolites.get_by_id('fad_c')
fe2_c=model.metabolites.get_by_id('fe2_c')
fe3_c=model.metabolites.get_by_id('fe3_c')
k_c=model.metabolites.get_by_id('k_c')
mg2_c=model.metabolites.get_by_id('mg2_c')
mlthf_c=model.metabolites.get_by_id('mlthf_c')
mn2_c=model.metabolites.get_by_id('mn2_c')
mobd_c=model.metabolites.get_by_id('mobd_c')
nad_c=model.metabolites.get_by_id('nad_c')
nadp_c=model.metabolites.get_by_id('nadp_c')
nh4_c=model.metabolites.get_by_id('nh4_c')
pheme_c=model.metabolites.get_by_id('pheme_c')
pydx5p_c=model.metabolites.get_by_id('pydx5p_c')
ribflv_c=model.metabolites.get_by_id('ribflv_c')
sheme_c=model.metabolites.get_by_id('sheme_c')
so4_c=model.metabolites.get_by_id('so4_c')
thf_c=model.metabolites.get_by_id('thf_c')
thmpp_c=model.metabolites.get_by_id('thmpp_c')
udcpdp_c=model.metabolites.get_by_id('udcpdp_c')
zn2_c=model.metabolites.get_by_id('zn2_c')
h_c=model.metabolites.get_by_id('h_c')
pi_c=model.metabolites.get_by_id('pi_c')
adp_c=model.metabolites.get_by_id('adp_c')
biomass=model.metabolites.get_by_id('biomass')
capsule_NM_e=model.metabolites.get_by_id('capsule_NM_e')
dna_c=model.metabolites.get_by_id('dna_c')
rna_c=model.metabolites.get_by_id('rna_c')
pepglycan_p=model.metabolites.get_by_id('pepglycan_p')
protein=model.metabolites.get_by_id('protein')
lipid_NM_p=model.metabolites.get_by_id('lipid_NM_p')
ptrc_c=model.metabolites.get_by_id('ptrc_c')


In [265]:
#remove Nm_biomass components
r=model.reactions.Nm_biomass
r.add_metabolites({
    fthf_c:0.000223,
    ohph_c:0.000223,
    amet_c:0.000223,
    atp_c:52.84,
    ca2_c:0.004737,
    capsule_NM_e:0.00146,
    cl_c:0.004737,
    coa_c:0.000576,
    cobalt2_c:0.003158,
    cu2_c:0.003158,
    dna_c:0.0798,
    fad_c:0.000223,
    fe2_c:0.007106,
    fe3_c:0.007106,
    h2o_c:52.84,
    k_c:0.1776,
    lipid_NM_p:0.1483,
    mg2_c:0.007895,
    mlthf_c:0.000223,
    mn2_c:0.003158,
    mobd_c:0.003158,
    nad_c:0.001831,
    nadp_c:0.000447,
    nh4_c:0.011843,
    pepglycan_p:0.1389,
    pheme_c:0.000223,
    protein:5.55,
    ptrc_c:0.0077376,
    pydx5p_c:0.000223,
    ribflv_c:0.000223,
    rna_c:0.4432,
    sheme_c:0.000223,
    so4_c:0.003948,
    thf_c:0.000223,
    thmpp_c:0.000223,
    udcpdp_c:0.000055,
    zn2_c:0.003158,
    h_c:-52.84,
    pi_c:-52.84,
    adp_c:-52.84,
    biomass:-1,})



In [266]:
#New biomass equation for Gc

r=model.reactions.Nm_biomass
r.add_metabolites({
    pa120_p:-0.000503555042155509,
    pa140_p:-0,
    pa160_p:-0.00207597350252432,
    pa180_p:-0.000290080640591335,
    pa141_p:-0.000371731800974898,
    pa161_p:-0.00564538392418283,
    pa181_p:-0.00216477129604764,
    pe120_p:-0.00234345993674653,
    pe140_p:-0.0025598346977071,
    pe160_p:-0.055748823259178,
    pe180_p:-0.00137834776769998,
    pe141_p:-0.00821977537556325,
    pe161_p:-0.0255682154771119,
    pe181_p:-0.0102822273478348,
    pg120_p:-0.000703620745734723,
    pg140_p:-3.03207570481577E-05,
    pg160_p:-0.0122303684865845,
    pg180_p:-0.000418944804169265,
    pg141_p:-0.00290595573235281,
    pg161a_p:-0.00862596402374158,
    pg181c_p:-0.00312454317572049,
    los_GC_c:-0.0129924495638587,
    murein4px4px4pOAc_p:-0.005556,
    murein4p4p4pOAc_p:-0.008334,
    asp_L_c:-0.328506488951245,
    glu_L_c:-0.417926718513411,
    ser_L_c:-0.298519204391151,
    his_L_c:-0.101130783859868,
    gly_c:-0.63200279142024,
    thr_L_c:-0.289017233514768,
    ala_L_c:-0.801362443841416,
    arg_L_c:-0.326223794247415,
    tyr_L_c:-0.142730093324994,
    val_L_c:-0.43926986171134,
    met_L_c:-0.102311261238582,
    phe_L_c:-0.199024522901412,
    ile_L_c:-0.291900141026265,
    leu_L_c:-0.472627376778854,
    lys_L_c:-0.351058182823571,
    pro_L_c:-0.246449661470245,
    asn_L_c:-0.251609961417047,
    trp_L_c:-0.0689480779843109,
    cys_L_c:-0.0606915932106025,
    gln_L_c:-0.246224295606094,
    datp_c:-0.0193660372420178,
    dttp_c:-0.0193660372420178,
    dgtp_c:-0.0205433507259598,
    dctp_c:-0.0205433507259598,
    atp_c:-0.108889121528625,
    utp_c:-0.108889121528625,
    gtp_c:-0.112753363956914,
    ctp_c:-0.112753363956914,
    fthf_c:-0.000223,
    ohph_c:-0.000223,
    amet_c:-0.000223,
    ca2_c:-0.004737,
    cl_c:-0.004737,
    coa_c:-0.000576,
    cobalt2_c:-0.003158,
    cu2_c:-0.003158,
    fad_c:-0.000223,
    fe2_c:-0.007106,
    fe3_c:-0.007106,
    k_c:-0.1776,
    mg2_c:-0.007895,
    mlthf_c:-0.000223,
    mn2_c:-0.003158,
    mobd_c:-0.003158,
    nad_c:-0.001831,
    nadp_c:-0.000447,
    nh4_c:-0.11843,
    pheme_c:-0.000223,
    pydx5p_c:-0.000223,
    ribflv_c:-0.000223,
    sheme_c:-0.000223,
    so4_c:-0.003948,
    thf_c:-0.000223,
    thmpp_c:-0.000223,
    udcpdp_c:-0.000055,
    zn2_c:-0.003158,
    atp_c: -52.84,
    h_c:52.84,
    pi_c:52.84,
    adp_c:52.84,
    biomass:1,
    })
model.slim_optimize()

r.name='biomass'
r.id='biomass'
model.objective='biomass'
model.slim_optimize()

2.997637024061464

In [267]:
rpmi(model)
print ('RPMI',model.slim_optimize())
mdm(model)
print ('MDM', model.slim_optimize())
complete(model)
print('Complete media', model.slim_optimize())

RPMI 1.6424778652469958
MDM 1.5275729448161122
Complete media 2.9976370240617043


In [268]:
model.remove_reactions('PLIPIDS')
model.remove_reactions('PLIPIDS_ESS')
model.remove_reactions('PEPGLY')
model.remove_reactions('PROTS')
model.remove_reactions('DNAS')
model.remove_reactions('RNAS')
model.remove_reactions('Nm_Ess_biomass')
model.remove_reactions('Nm_Baart_biomass')
model.remove_reactions('DM_CAPBIOMASS')
model.remove_reactions('DM_ESSBIOMASS')

C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [269]:
model.metabolites.trp_L_c.annotation['bigg.metabolite'] = 'trp__L'

In [270]:
#Add exchange reactions for extracellular reactions that are missing, or delete metabolites as appropriate
exchanges=[]
exchanges_mets=[]
extracellular=[]
for metabolite in model.metabolites:
    if metabolite.compartment == 'extracellular':
        extracellular.append(metabolite.id)
    else :
        pass

for reaction in model.reactions:
    if "EX_" in reaction.id:
        exchanges.append(reaction.id)

for x in exchanges:
    for metabolites in model.reactions.get_by_id(x).metabolites:
        exchanges_mets.append(metabolites.id)

In [271]:
for x in extracellular:
    if x in exchanges_mets:
        pass
    else:
        print(x)

ag_e
capsule_NM_e
glcn_e
acald_e


In [272]:
#acald export was previously deleted as it is not detected extracellularly in Gc cultures. removed extracellular acald as a metabolite.  
acald_e = model.metabolites.acald_e
model.remove_metabolites(acald_e)
ACALDtex=model.reactions.ACALDtex
model.remove_reactions(ACALDtex)

#gluconate is not thought to be imported/used as a carbon source by neisseria (https://www.karger.com/Article/FullText/447093)
glcn_e=model.metabolites.glcn_e
model.remove_metabolites(glcn_e)

#delete capsule metabolite, which is not produced by Gc
capsule=model.metabolites.capsule_NM_e
model.remove_metabolites(capsule)

#add exchange reaction for silver
reaction = Reaction('EX_ag_e')
reaction.name = 'EX of Ag_e'
reaction.subsystem = 'Exchange'
reaction.lower_bound = -1000
reaction.upper_bound = 1000
reaction.annotation = {'sbo': 'SBO:0000627'}
reaction.add_metabolites({model.metabolites.ag_e: -1.0})
reaction.notes['NOTES'] = 'Added exchange reaction that was missing'
model.add_reactions([reaction])

In [273]:
#delete orphan metabolties left over from reconstruction curation process
orphan_mets=[]
for metabolite in model.metabolites:
    if len(metabolite.reactions) == 0:
        orphan_mets.append(metabolite.id)

for mets in orphan_mets:
    m=model.metabolites.get_by_id(mets)
    model.remove_metabolites(m)

In [274]:
pro_L_c

Metabolite identifier,pro_L_c
Name,L-Proline
Memory address,0x020f45ae7280
Formula,C5H9NO2
Compartment,cytosol
In 7 reaction(s),"PROHX, PROD2, PROt2rpp, P5CR, biomass, PROabcpp, PROt1pp"


In [275]:
r=model.reactions.AMPTASEPG
pro__L_c = model.metabolites.pro__L_c

r.add_metabolites({
    pro_L_c: 1.0,
    pro__L_c: -1.0})


r

Reaction identifier,AMPTASEPG
Name,Aminopeptidase (pro-gly)
Memory address,0x020f4c8127f0
Stoichiometry,h2o_c + progly_c --> gly_c + pro_L_c H2O + L-Prolinylglycine --> Glycine + L-Proline
GPR,NGO1227
Lower bound,0.0
Upper bound,1000.0


In [276]:
model.remove_metabolites(pro__L_c)

## 2d. Remove loops

In [277]:
#To remove an infeasible loop, where gln is exchanged infinitely between the two Na+ and h+ dependent transporters, limited these to import only.

model.reactions.GLNt2pp.lower_bound =0
model.reactions.GLNt1pp.lower_bound =0  

In [278]:
#D_ala import is predicted to occur through an orphan reaction and spontaneously
#Linking D_ala import to L_ala import genetically and na+ and h+ according to the transporters as appropriate


ALADt2pp =model.reactions.ALADt

ALADt2pp.id = 'ALADt2pp'
ALADt2pp.gene_reaction_rule = 'NGO1807 or NGO1787 or NGO1290 or NGO1957'

h_p = model.metabolites.get_by_id('h_p')
h_c = model.metabolites.get_by_id('h_c')
na1_c = model.metabolites.get_by_id('na1_c')
na1_p = model.metabolites.get_by_id('na1_p')
ala_D_p = model.metabolites.get_by_id('ala_D_p')
ala_D_c = model.metabolites.get_by_id('ala_D_c')

ALADt2pp.add_metabolites({
        na1_p: -1.0,
        na1_c: 1.0
        })

print(ALADt2pp.check_mass_balance())

ALADt1pp = cobra.Reaction('ALADt1pp')
ALADt1pp.name = 'D_ala import'
ALADt1pp.gene_reaction_rule = 'NGO1290 or NGO1787'
ALADt1pp.lower_bound = -1000.
ALADt1pp.upper_bound = 1000.
ALADt1pp.add_metabolites({
    ala_D_p: -1.0,
    h_p: -1.0,
    ala_D_c: 1.0,
    h_c: 1.0
    })


print(ALADt1pp.check_mass_balance())


{}
{}


In [279]:
#Remove looping reactions
model.reactions.ALAt1pp.lower_bound = 0
model.reactions.ALAt2pp.lower_bound = 0
model.reactions.ARGt1pp.lower_bound = 0
model.reactions.ARGt2pp.lower_bound = 0
model.reactions.ASNt1pp.lower_bound = 0
model.reactions.ASNt2pp.lower_bound = 0
model.reactions.ASPt1pp.lower_bound = 0
model.reactions.ASPt2pp.lower_bound = 0
model.reactions.CYSt1pp.lower_bound = 0
model.reactions.CYSt2pp.lower_bound = 0
model.reactions.GLUt1pp.lower_bound = 0
model.reactions.GLUt2pp.lower_bound = 0
model.reactions.HISt1pp.lower_bound = 0
model.reactions.HISt2pp.lower_bound = 0
model.reactions.ILEt1pp.lower_bound = 0
model.reactions.ILEt3pp.lower_bound = 0
model.reactions.LEUt1pp.lower_bound = 0
model.reactions.LEUt3pp.lower_bound = 0
model.reactions.LYSt1pp.lower_bound = 0
model.reactions.LYSt2pp.lower_bound = 0
model.reactions.METt1pp.lower_bound = 0
model.reactions.METt2pp.lower_bound = 0
model.reactions.PHEt1pp.lower_bound = 0
model.reactions.PHEt2pp.lower_bound = 0
model.reactions.PROt1pp.lower_bound = 0
model.reactions.PROt2rpp.lower_bound = 0
model.reactions.SERt1rpp.lower_bound = 0
model.reactions.SERt2pp.lower_bound = 0
model.reactions.THRt1rpp.lower_bound = 0
model.reactions.THRt2pp.lower_bound = 0
model.reactions.VALt1pp.lower_bound = 0
model.reactions.VALt3pp.lower_bound = 0
model.reactions.TRPS1.lower_bound=0
model.reactions.ADK1.lower_bound=0
model.reactions.NDPK1.lower_bound=0
model.reactions.VALTA.upper_bound=0
model.reactions.VPAMT.upper_bound=0

In [280]:
#This gene catalyzes a bidirection conversion of alanine into pyruvate. However, although the enzyme is technically birectional,
#in e.coli alanine transaminase expretion is supressed in the presence of alanine and the downstream product leucine. 
#Additionally, Gc is not capable of growing on alanine as a carbon source. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2950514/
#As such, this is likely not a catabolic enzyme. I am limiting it to the synthesis of alanine only. 
model.reactions.ALATA_L.upper_bound=0

In [281]:
rpmi(model)
print ('RPMI',model.slim_optimize())
mdm(model)
print ('MDM', model.slim_optimize())
complete(model)
print('Complete media', model.slim_optimize())

RPMI 1.5748164611774687
MDM 1.4529034642314562
Complete media 2.4998508320954755


In [282]:
for metabolite in model.metabolites:
    if 'Cytosol' in metabolite.compartment:
        metabolite.compartment == 'cytosol'
    if '_c' in metabolite.id:
        metabolite.compartment == 'cytosol'
    if '_e' in metabolite.id:
        metabolite.compartment == 'extracellular'
    if '_p' in metabolite.id:
        metabolite.compartment == 'periplasm'

In [283]:
model.metabolites.dms_p.compartment

'periplasm'

In [284]:
for metabolite in model.metabolites:
    if '' in metabolite.compartment:
        print(metabolite.id)

dms_p
pg161_c
12dgr181_p
3hcmrs7eACP_c
12dgr181_c
dms_e
dhor_S_c
2agpe140_p
dttp_c
thym_e
spmd_c
f6p_e
f6p_c
adp_c
aso3_p
aso3_e
lac_D_p
lyx_L_p
gcald_c
2ombzl_c
3hmrsACP_c
lac_D_c
lac_D_e
lyx_L_e
mal_L_p
galicolipa_c
pe160_p
pg161_p
mal_L_c
mal_L_e
lgt_S_c
pe160_c
2ommbl_c
hcys_L_c
glcur1p_p
anth_c
cmp_e
anhgm4p_p
5dglcn_p
methf_c
ps161_c
anhgm4p_c
thf_c
ade_c
rhcys_c
ade_e
ps120_c
ade_p
h2o2_p
tyr_L_p
4h2oglt_c
h2o2_e
din_e
h2o2_c
tyr_L_c
tyr_L_e
ahdt_c
ala_L_c
ala_L_e
3php_c
cmrs3eACP_c
cdp161_c
ala_L_p
murein5p5p5p_p
n2o_p
thm_p
u3hga_c
thm_c
thm_e
thym_p
acon_C_c
cit_p
hxan_c
hxan_e
progly_e
cit_c
hxan_p
val_L_c
val_L_e
udpg_c
murein4px4px4p_p
udpg_e
val_L_p
udpg_p
ACP_c
hmbil_c
udpacgal_p
3hhexACP_c
udpacgal_e
gnagalicolipa_c
3mob_c
iasp_c
LalaDgluMdap_p
fmn_c
LalaDgluMdap_c
LalaDgluMdap_e
ac_c
ac_e
lald_L_c
no3_p
ac_p
ocACP_c
no3_e
ag_e
ag_c
h2s_p
cu2_p
glx_c
2oph_c
h2s_e
h2s_c
1odecg3p_c
cu2_c
cu2_e
2agpg140_p
tmao_e
2agpg140_c
dtdp_c
tmao_p
2agpg180_c
cys_D_p
succ_p
glu5sa_c
s

In [285]:
len(model.genes)
#Subtract 2 for "Sgene, and orphan" 

559

In [286]:
model.id='iNGO_557'

In [287]:
model

Name,iNGO_557
Memory address,0x020f3d096130
Number of metabolites,1263
Number of reactions,1424
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"periplasm, cytosol, extracellular, Cytosol"


## Add Model Annotations

In [288]:
#Add Kegg annotations
for gene in model.genes:
    gene.annotation['kegg.genes']='ngo:'+ str(gene)

In [289]:
#load gene annotation table
genes= list()
for gene in model.genes:
    genes.append(gene.id)
genes

#Use PATRIC/BVBRC database, uniprot database, and PubMLST
annotation_table = pd.read_csv (cwd+'/Gc_GENRE_2022/Curate_Gc_Model/annotation_resources/refseq_gene_annotations.csv', usecols= ['RefSeq_ID', 'Protein_ID', 'Gene_ID', 'Uniprot_ID', 'BVBRC_ID','PUBMLST_locus' ],index_col=0)
annotation_table = annotation_table.fillna('')
annotation_table                       


,Protein_ID,Gene_ID,Uniprot_ID,BVBRC_ID,PUBMLST_locus
RefSeq_ID,,,,,
NGO0001,YP_207186.1,3283049,Q5FAJ2,fig|242231.10.peg.1,NEIS0320
NGO0002,YP_207187.1,3283050,Q5FAJ1,fig|242231.10.peg.2,NEIS0321
NGO0003,YP_207188.1,3283051,Q5FAJ0,fig|242231.10.peg.3,NEIS0323
NGO0004,YP_207189.1,3283052,,fig|242231.10.peg.4,NEIS0324
NGO0006,YP_207190.1,3283053,,fig|242231.10.peg.7,NEIS0326
...,...,...,...,...,...
NGO2178,YP_209183.1,3282747,Q5F4W6,fig|242231.10.peg.2631,NEIS0316
NGO2179,YP_209184.1,3282746,Q5F4W5,fig|242231.10.peg.2632,
NGO2180,YP_209185.1,3282745,Q5F4W4,fig|242231.10.peg.2633,NEIS0317


In [290]:
#Add gene annotations to model from annotation table
for x in model.genes:
    geneid=x.id
    if geneid in annotation_table.index:
        protein_id=annotation_table['Protein_ID'].loc[geneid]
        model.genes.get_by_id(geneid).annotation['ncbiprotein']=protein_id
        ncbigene_id=annotation_table['Gene_ID'].loc[geneid]
        model.genes.get_by_id(geneid).annotation['ncbigene']= str(ncbigene_id)
        uniprot_id=annotation_table['Uniprot_ID'].loc[geneid]
        model.genes.get_by_id(geneid).annotation['UNIPROT']= uniprot_id
        patric_id=annotation_table['BVBRC_ID'].loc[geneid]
        model.genes.get_by_id(geneid).annotation['PATRIC']=patric_id
        pubmlst_id=annotation_table['PUBMLST_locus'].loc[geneid]
        model.genes.get_by_id(geneid).annotation['PUBMLST_Locus']=pubmlst_id
    else:
        pass

In [291]:
# Add SBOs

# Metabolites
for cpd in model.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'

# Reactions
for rxn in model.reactions:
    substrates = list(rxn.metabolites)
    compartments = set([x.compartment for x in substrates])
    
    if 'EX_' in rxn.id:
        rxn.annotation['sbo'] = 'SBO:0000627' # exchange
    elif len(compartments) > 1:
        rxn.annotation['sbo'] = 'SBO:0000185' # transport
    else:
        rxn.annotation['sbo'] = 'SBO:0000176' # metabolic

# Biomass
model.reactions.biomass.annotation['sbo'] = 'SBO:0000629'  

#Demand reactions
model.reactions.DM_BIOMASS.annotation['sbo'] = 'SBO:0000628'  
model.reactions.DM_4HBA.annotation['sbo'] = 'SBO:0000628'  
model.reactions.DM_5DRIB.annotation['sbo'] = 'SBO:0000628' 
model.reactions.DM_AMOB.annotation['sbo'] = 'SBO:0000628'  
model.reactions.DM_BIOMASS.annotation['sbo'] = 'SBO:0000628'  
model.reactions.DM_BV.annotation['sbo'] = 'SBO:0000628'  
model.reactions.DM_HMFURN.annotation['sbo'] = 'SBO:0000628'
model.reactions.DM_PAP.annotation['sbo'] = 'SBO:0000628'
model.reactions.DM_UREA.annotation['sbo'] = 'SBO:0000628'


# Genes
for gene in model.genes:
    gene.annotation['sbo'] = 'SBO:0000243'

In [292]:
model.genes.NGO1881.annotation

{'kegg.genes': 'ngo:NGO1881',
 'ncbiprotein': 'YP_208914.1',
 'ncbigene': '3282312',
 'UNIPROT': 'Q5F5N5',
 'PATRIC': 'fig|242231.10.peg.2264',
 'PUBMLST_Locus': 'NEIS0074',
 'sbo': 'SBO:0000243'}

In [293]:
polisher=read_model(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_557_polished.sbml')

https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/UNIPROT/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/PUBMLST_Lo

In [294]:
for rxn in model.reactions:
    if rxn.id in polisher.reactions:
        rxn.annotation = polisher.reactions.get_by_id(str(rxn.id)).annotation

In [295]:
for met in model.metabolites:
    if met.id in polisher.metabolites:
        met.annotation = polisher.metabolites.get_by_id(str(met.id)).annotation

## Save Finalized Model

In [296]:
cobra.io.save_json_model(model, cwd+'/Gc_GENRE_2022/Models/NGO_557.json')

In [297]:
cobra.io.write_sbml_model(model, cwd+'/Gc_GENRE_2022/Models/NGO_557.sbml')

In [298]:
model

Name,iNGO_557
Memory address,0x020f3d096130
Number of metabolites,1263
Number of reactions,1424
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"periplasm, cytosol, extracellular, Cytosol"
